In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
# plt.rcParams["font.family"] = "serif"
# plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=False
yu.slName='plots_notes_NST'

In [ ]:
ensembles=['cA211.530.24','cA2.09.48']

ens2data={}
for ens in ensembles:
    path={
        'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_c/data/cA211.530.24_NST_a_daint.h5',
        'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_c/data/cA2.09.48_NST_c_booster.h5',
        }[ens]
    ens2data[ens]=yu.load(path)
    print('Ncfg='+str(yu.deepKey(ens2data[ens]['2pt'],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data['2pt'],2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

In [ ]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and conjugated 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'r2pt': True, # making 2ptMat real for the rest frame # 'll2pt' has to be real for this flag
    'remove_pi0VEV':True, 
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    res=np.zeros([Ncfg,Tpack],dtype=complex)
    if opa+'_'+opb in data['2pt']:
        t=data['2pt'][opa+'_'+opb]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt']:
        t=data['2pt'][opb+'_'+opa]; 
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0))
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        if ops[0].split(';')[1]=='0,0,0' and flags['r2pt']:
            t=np.real(t)
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    res=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)
    if opa+'_'+opb in data['3pt']:
        t=data['3pt'][opa+'_'+opb][insert]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}
        if opa+'_'+opb not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opa+'_'+opb][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

def remove_pi0VEV(Ctij,pi0VEV,opas,opbs):
    if not flags['remove_pi0VEV']:
        return Ctij
    
    factor=yu.coeff['g;0,0,1;G1;N1pi0,a;l1;N,pi'][0][1]
    
    ka=None # index for N(n)pi0(0)
    for i,op in enumerate(opas):
        if yu.getNpar(op)==1:
            qa=i # index for N(n)
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            ka=i
            factor_a = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
    kb=None
    for i,op in enumerate(opbs):
        if yu.getNpar(op)==1:
            qb=i
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            kb=i
            factor_b = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
            
    res=Ctij.copy()
    if ka is not None:
        res[:,ka,:]-=factor_a*Ctij[:,qa,:]
    if kb is not None:
        res[:,:,kb]-=np.conj(factor_b)*Ctij[:,:,qb]
    if ka is not None and kb is not None:
        res[:,ka,kb]+=factor_a*np.conj(factor_b)*Ctij[:,qa,qb]
        
    return res

pt0='0,0,0'; ofs0=[('a','p'),('N1pi1,a','12'),]; ops0=yu.getops(pt0,'l1',ofs0)
pt1='0,0,1'; ofs1=[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]; ops1=yu.getops(pt1,'l1',ofs1)

L2ens={'24':'cA211.530.24','48':'cA2.09.48'}
L2T_ens={'24':'cA211.530.24','48':'cA2.09.48'}
ensLs=['24','48']

L2ops={'0':ops0,'1':ops1}
L2ptofs={'0':(pt0,ofs0),'1':(pt1,ofs1)}
L2T_pt={'0':r'$\vec{p}=(0,0,0)$','1':r'$\vec{p}=(0,0,1)$'}

L2diags={'noj':yu.diags_jLoopless,'all':yu.diags_all}
L2T_diags={'noj':'no j-loop','all':'all diags'}
diagsLs=['noj','all']

In [ ]:
# imaginary part

updateEns(ensembles[1])
xUnit=lat.a

fig, axs = yu.getFigAxs(2,2,sharex=True)

irow=0
for icol,ri in enumerate(['Real','Imag']):
    func_ri=np.real if ri in ['Real'] else np.imag
    for l in ['l1','l2']:
        color={'l1':'r','l2':'b'}[l]
        mfc={'l1':None,'l2':'white'}[l]
        opa=yu.getop('0,0,1',l,('a','p')); opb=yu.getop('0,0,1',l,('N0pi1,a','12'))
        dat=get2pt(opa,opb)
        def func(dat):
            t=np.mean(dat,axis=0)
            return func_ri(t)
        mean,err,cov=yu.jackknife(dat,func)
        
        tmin=1; tmax=20
        tmean=mean[0][tmin:tmax]; terr=err[0][tmin:tmax]; shift={'l1':-0.1,'l2':0.1}[l]
        axs[irow,icol].errorbar((np.arange(tmin,tmax)+shift)*xUnit,tmean,terr,color=color,fmt='s',mfc=mfc,label=\
            {'l1':r'$r=0$','l2':r'$r=1$'}[l])
        
irow=1
for icol,ri in enumerate(['Real','Imag']):
    func_ri=np.real if ri in ['Real'] else np.imag
    for l in ['l1','l2']:
        color={'l1':'r','l2':'b'}[l]
        mfc={'l1':None,'l2':'white'}[l]
        opa=yu.getop('0,0,1',l,('a','p')); opb=yu.getop('0,0,1',l,('N1pi0,a','12'))
        dat=get2pt(opa,opb)
        def func(dat):
            t=np.mean(dat,axis=0)
            return func_ri(t)
        mean,err,cov=yu.jackknife(dat,func)
        
        tmin=1; tmax=20
        tmean=mean[0][tmin:tmax]; terr=err[0][tmin:tmax]; shift={'l1':-0.1,'l2':0.1}[l]
        axs[irow,icol].errorbar((np.arange(tmin,tmax)+shift)*xUnit,tmean,terr,color=color,fmt='s',mfc=mfc,label=\
            {'l1':r'$r=0$','l2':r'$r=1$'}[l])

axs[0,0].legend()
axs[0,0].set_title(r'Real',fontsize=40)
axs[0,1].set_title(r'Imag',fontsize=40)

axs[0,0].set_ylabel(r'$N(1)\to N(0)\pi(1)$',fontsize=40)
axs[1,0].set_ylabel(r'$N(1)\to N(1)\pi(0)$',fontsize=40)

axs[1,0].set_xlabel(r'$t$ [fm]',fontsize=40)
axs[1,1].set_xlabel(r'$t$ [fm]',fontsize=40)

for irow in range(2):
    for icol in range(2):
        axs[irow,icol].locator_params(axis='x', nbins=4)
        axs[irow,icol].locator_params(axis='y', nbins=3)
        axs[irow,icol].tick_params(axis='both', which='major', labelsize=40) 

plt.tight_layout()
plt.savefig('fig/ccl1l2.pdf')
plt.close()

In [ ]:
# isospin

updateEns(ensembles[1])
xUnit=lat.a

fig, axs = yu.getFigAxs(1,1)

pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0')]
# pt='0,0,1'; ofs=[('a','p'),('N0pi1,a','n,pi+'),('N0pi1,a','p,pi0')]
# pt='0,0,1'; ofs=[('a','p'),('N1pi0,a','n,pi+'),('N1pi0,a','p,pi0')]
ops=yu.getops(pt,'l1',ofs)
get2ptMat(ops)
dat=[get2ptMat(ops),get2ptMat(ops,diags=yu.diags_loopless),data['VEV']['pi0f']]

def func(dat):
    t=yu.meanDeep(dat)
    # t[0]=remove_pi0VEV(t[0],t[-1],ops,ops)
    return [(-np.sqrt(2))*np.real(t[1][:,0,2]),np.real(t[0][:,0,1]),(-np.sqrt(2))*np.real(t[0][:,0,2])]

mean,err,cov=yu.jackknife(dat,func)

irow=0; icol=0
ts=np.arange(9,15+1)
tmean=mean[1][ts]; terr=err[1][ts]; shift=-0.15
axs[irow,icol].errorbar(ts*xUnit+shift*xUnit,tmean,terr,color='r',fmt='o',label=r'$\langle \mathcal{J}_{p}\;\mathcal{J}^\dagger_{n\pi^+}\rangle$')

tmean=mean[0][ts]; terr=err[0][ts]; shift=0
axs[irow,icol].errorbar(ts*xUnit+shift*xUnit,tmean,terr,mfc='white',color='b',label=r'$\langle \mathcal{J}_{p}\;\mathcal{J}^\dagger_{p\pi^0}\rangle_{\mathrm{conn}} *(-\sqrt{2})$')

tmean=mean[2][ts]; terr=err[2][ts]; shift=0.15
axs[irow,icol].errorbar(ts*xUnit+shift*xUnit,tmean,terr,color='b',label=r'$\langle \mathcal{J}_{p}\;\mathcal{J}^\dagger_{p\pi^0}\rangle_{\mathrm{full}} *(-\sqrt{2})$')

axs[irow,icol].legend(fontsize=15)

axs[0,0].set_xlabel(r'$t$ [fm]')
axs[0,0].set_ylabel(r'2-point function')

plt.tight_layout()
plt.savefig('fig/symmetry_isospin.pdf')
plt.close()

In [ ]:
# t0dep

updateEns(ensembles[1])

fig, axs = yu.getFigAxs(1,2,sharex=True)
# yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors'])
# yu.addColHeader(axs,[r'$t_0=0.1\,$fm'.format(t0),r'$t-t_0=0.5\,$fm'])

t0s=[2,7,12]
for t0,color,shift in zip(t0s,['r','g','b'],[-0.2,0.,0.2]):
    pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))  # if tRef>=0 else np.log(eVals)/tRef
        return En.T
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a; yUnit=lat.aInv/1000
    ts=np.array([t for t in range(t0+1,20)])
    tmean=mean[0][ts]; terr=err[0][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color=color,label='1'+str(t0))
    ts=np.array([t for t in range(t0+1,14)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color=color,fmt='d',label='2'+str(t0))
axs[0,0].set_xlim([0.2,1.95])
axs[0,0].set_xlabel(r'$t$ [fm]',fontsize=20)
axs[0,0].set_ylabel(r'$E_{\mathrm{eff}}$ [GeV]',fontsize=20)
    
handles, _ = axs[0,0].get_legend_handles_labels()
axs[0,0].legend([handles[idx] for idx in [0,1,0,2,4]],[r'$E_N$',r'$E_{1}$']+[r'$t_0/a={}$'.format(t0) for t0 in t0s],fontsize='xx-large')

for t0,color,fmt in [(1,'b','s'),(3,'slateblue','d'),(5,'blueviolet','o')]:
    pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        eVecs=yu.renormalize_eVecs(eVecs)
        t=eVecs[:,0,:].T
        t=t/t[0,:]
        return eV_func(t)
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a
    ts=np.array([t for t in range(t0+1,20)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,1].errorbar(ts*xUnit-0.01,tmean,terr,color=color,mfc='white',fmt=fmt,label=r'$t_0/a={}$'.format(t0))


dtt0=4
def func(dat):
    t=yu.meanDeep(dat)
    t2pt,tVEVpi0f=t
    t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
    (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
    eVecs=yu.renormalize_eVecs(eVecs)
    t=eVecs[:,0,:].T
    t=t/t[0,:]
    return eV_func(t)
mean,err,cov=yu.jackknife(dat,func)

xUnit=lat.a
ts=np.array([t for t in range(dtt0+1,18)])
tmean=mean[1][ts]; terr=err[1][ts]
axs[0,1].errorbar(ts*xUnit+0.01,tmean,terr,color='r',mfc=None,label=r'$(t-t_0)/a={}$'.format(dtt0))

axs[0,1].set_ylim([0.,0.05])
axs[0,1].set_xlabel(r'$t$ [fm]',fontsize=20)
axs[0,1].set_ylabel(r'Eigenvector',fontsize=20)
axs[0,1].legend(loc='upper left',fontsize='xx-large')

axs[0,0].locator_params(axis='x', nbins=4)
axs[0,0].tick_params(axis='both', which='major', labelsize=20) 
axs[0,1].locator_params(axis='y', nbins=4)
axs[0,1].tick_params(axis='both', which='major', labelsize=20) 

# axs[0,1].text(.05, .95, r'$v_{N,O_{N\pi}}$', ha='left', va='top', size=20, transform=axs[0,0].transAxes)
# axs[0,1].text(.05, .85, r'$t_0=0.1\,$fm', ha='left', va='top', size=20, transform=axs[0,0].transAxes)
# axs[0,1].text(.05, .85, r'$t-t_0=0.4\,$fm', ha='left', va='top', size=20, transform=axs[0,1].transAxes)


plt.tight_layout()
plt.savefig('fig/t0dep.pdf')
plt.close()

In [ ]:
# e-values 24

fsize=40

updateEns(ensembles[0])

fig, axs = yu.getFigAxs(1,2,sharex=True,sharey=True)
# yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors'])
yu.addColHeader(axs,[r'$\vec{p}=(0,0,0)$',r'$\vec{p}=(0,0,1)$'],fontsize=fsize)

t0s=[2]

for t0,color,shift in zip(t0s,['r','g','b'],[-0.2,0.,0.2]):
    pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))  # if tRef>=0 else np.log(eVals)/tRef
        return En.T
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a; yUnit=lat.aInv/1000
    ts=np.array([t for t in range(t0+1,15)])
    tmean=mean[0][ts]; terr=err[0][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='r',label='1'+str(t0))
    ts=np.array([t for t in range(t0+1,11)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='g',fmt='d',label='2'+str(t0))   

axs[0,0].set_xlim([0.2,1.95])
axs[0,0].set_ylim([1.0,2.5])
axs[0,0].set_yticks([1.2,1.7,2.2])
axs[0,0].set_xlabel(r'$t$ [fm]',fontsize=fsize)
axs[0,0].set_ylabel(r'$E_{\mathrm{eff}}$ [GeV]',fontsize=fsize)
    
handles, _ = axs[0,0].get_legend_handles_labels()
axs[0,0].legend([handles[idx] for idx in [0,1]],[r'$E_N$',r'$E_1$']+[r'$t_0/a={}$'.format(t0) for t0 in t0s],fontsize='xx-large')


axs[0,0].locator_params(axis='x', nbins=4)
axs[0,0].tick_params(axis='both', which='major', labelsize=fsize) 

for t0,color,shift in zip(t0s,['r','g','b'],[-0.2,0.,0.2]):
    pt='0,0,1'; ofs=[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))  # if tRef>=0 else np.log(eVals)/tRef
        return En.T
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a; yUnit=lat.aInv/1000
    ts=np.array([t for t in range(t0+1,15)])
    tmean=mean[0][ts]; terr=err[0][ts]
    axs[0,1].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='r',label='1'+str(t0))
    ts=np.array([t for t in range(t0+1,15)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,1].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='g',fmt='d',label='2'+str(t0))
    ts=np.array([t for t in range(t0+1,14)])
    tmean=mean[2][ts]; terr=err[2][ts]
    axs[0,1].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='b',fmt='d',label='2'+str(t0))
axs[0,1].set_xlim([0.2,1.4])
axs[0,1].set_xlabel(r'$t$ [fm]',fontsize=fsize)
# axs[0,1].set_ylabel(r'$E_{\mathrm{eff}}$ [GeV]',fontsize=20)
    
handles, _ = axs[0,1].get_legend_handles_labels()
axs[0,1].legend([handles[idx] for idx in [0,1,2]],[r'$E_N$',r'$E_1$',r'$E_2$']+[r'$t_0/a={}$'.format(t0) for t0 in t0s],fontsize='xx-large')


axs[0,1].locator_params(axis='x', nbins=4)
axs[0,1].tick_params(axis='both', which='major', labelsize=fsize) 

axs[0,0].axhline(y=lat.getaEN(0)*lat.aInv/1000,color='grey',linestyle = '--', marker='') 
axs[0,0].axhline(y=(lat.getaEN(1)+lat.getaEpi(1))*lat.aInv/1000,color='grey',linestyle = '--', marker='') 

axs[0,1].axhline(y=lat.getaEN(1)*lat.aInv/1000,color='grey',linestyle = '--', marker='') 
axs[0,1].axhline(y=(lat.getaEN(1)+lat.getaEpi(0))*lat.aInv/1000,color='grey',linestyle = '--', marker='') 
axs[0,1].axhline(y=(lat.getaEN(0)+lat.getaEpi(1))*lat.aInv/1000,color='grey',linestyle = '--', marker='') 


plt.tight_layout()
plt.savefig('fig/evalues24.pdf')
plt.close()

In [ ]:
# e-values 48

updateEns(ensembles[1])

fig, axs = yu.getFigAxs(1,2,sharex=True,sharey=True)
# yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors'])
yu.addColHeader(axs,[r'$\vec{p}=(0,0,0)$',r'$\vec{p}=(0,0,1)$'],fontsize=fsize)

t0s=[2]

for t0,color,shift in zip(t0s,['r','g','b'],[-0.2,0.,0.2]):
    pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))  # if tRef>=0 else np.log(eVals)/tRef
        return En.T
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a; yUnit=lat.aInv/1000
    ts=np.array([t for t in range(t0+1,20)])
    tmean=mean[0][ts]; terr=err[0][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='r',label='1'+str(t0))
    ts=np.array([t for t in range(t0+1,14)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='g',fmt='d',label='2'+str(t0))
axs[0,0].set_xlim([0.2,1.95])
axs[0,0].set_ylim([0.8,1.6])
axs[0,0].set_yticks([0.9,1.2,1.5])
axs[0,0].set_xlabel(r'$t$ [fm]',fontsize=fsize)
axs[0,0].set_ylabel(r'$E_{\mathrm{eff}}$ [GeV]',fontsize=fsize)
    
handles, _ = axs[0,0].get_legend_handles_labels()
axs[0,0].legend([handles[idx] for idx in [0,1]],[r'$E_N$',r'$E_1$']+[r'$t_0/a={}$'.format(t0) for t0 in t0s],fontsize='xx-large')


axs[0,0].locator_params(axis='x', nbins=4)
axs[0,0].tick_params(axis='both', which='major', labelsize=fsize) 

for t0,color,shift in zip(t0s,['r','g','b'],[-0.2,0.,0.2]):
    pt='0,0,1'; ofs=[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))  # if tRef>=0 else np.log(eVals)/tRef
        return En.T
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a; yUnit=lat.aInv/1000
    ts=np.array([t for t in range(t0+1,20)])
    tmean=mean[0][ts]; terr=err[0][ts]
    axs[0,1].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='r',label='1'+str(t0))
    ts=np.array([t for t in range(t0+1,17)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,1].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='g',fmt='d',label='2'+str(t0))
    ts=np.array([t for t in range(t0+1,15)])
    tmean=mean[2][ts]; terr=err[2][ts]
    axs[0,1].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color='b',fmt='d',label='2'+str(t0))
# axs[0,1].set_xlim([0.2,1.95])
axs[0,1].set_xlabel(r'$t$ [fm]',fontsize=fsize)
# axs[0,1].set_ylabel(r'$E_{\mathrm{eff}}$ [GeV]',fontsize=20)
    
handles, _ = axs[0,1].get_legend_handles_labels()
axs[0,1].legend([handles[idx] for idx in [0,1,2]],[r'$E_N$',r'$E_1$',r'$E_2$']+[r'$t_0/a={}$'.format(t0) for t0 in t0s],fontsize='xx-large')


axs[0,1].locator_params(axis='x', nbins=4)
axs[0,1].tick_params(axis='both', which='major', labelsize=fsize) 

axs[0,0].axhline(y=lat.getaEN(0)*lat.aInv/1000,color='grey',linestyle = '--', marker='') 
axs[0,0].axhline(y=(lat.getaEN(1)+lat.getaEpi(1))*lat.aInv/1000,color='grey',linestyle = '--', marker='') 

axs[0,1].axhline(y=lat.getaEN(1)*lat.aInv/1000,color='grey',linestyle = '--', marker='') 
axs[0,1].axhline(y=(lat.getaEN(1)+lat.getaEpi(0))*lat.aInv/1000,color='grey',linestyle = '--', marker='') 
axs[0,1].axhline(y=(lat.getaEN(0)+lat.getaEpi(1))*lat.aInv/1000,color='grey',linestyle = '--', marker='') 

plt.tight_layout()
plt.savefig('fig/evalues48.pdf')
plt.close()

In [ ]:
# choices

dtt0s=range(1,11)
def run_t0_dep2(pt,ofs,diags=yu.diags_all,ylim1=None,ylim2=None,ylim3=None,ylim4=None,ylim5=None,label=None):
    Nop=len(ofs)
    
    fig, axs = yu.getFigAxs(3*2,len(dtt0s),sharex=True,sharey='row')
    yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors','cA24: W','cA48: Energy','cA48: E-vectors','cA48: W'])
    yu.addColHeader(axs,['dt='+str(dtt0) for dtt0 in dtt0s])
    
    eV_func=np.abs
    W_func=np.abs
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops,diags),data['VEV']['pi0f']]
        
        for i_dtt0,dtt0 in enumerate(dtt0s):
            irow=i_ens*3+1; icol=i_dtt0
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dtt0,tList=None if dtt0>=4 else range(Tpack-3))
                eVecs=yu.renormalize_eVecs(eVecs)
                t=eVecs[:,0,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func,sl_key=('t0_dep2'+label,ens,dtt0,'eVec'))
            for ind in range(Nop):
                tmin=dtt0+1; tmax=len(mean[ind])
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
            axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
            
            irow=i_ens*3+2
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dtt0,tList=None if dtt0>=4 else range(Tpack-3))
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func,sl_key=('t0_dep2'+label,ens,dtt0,'W'))
            for ind in range(0,1):
                tmin=dtt0+1; tmax=len(mean[ind])
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
            axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])
        
            
    plt.tight_layout()
    plt.savefig('fig/t0def_dtt0s'+label+'.pdf')
    plt.close()
    print(label)

# run_t0_dep2('0,0,0',[('a','p'),('N1pi1,a','12'),],ylim2=[0,0.5],ylim3=[0,0.05],ylim4=[0,0.4],ylim5=[0,0.2],label='_0_N1pi1')
# run_t0_dep2('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),],ylim2=[0,0.1],ylim3=[0,0.025],ylim4=[0,0.07],ylim5=[0,0.04],label='_1_N1pi0_N0pi1')

In [ ]:
# 0sub

dtt0=4

# yu.sl_reset()
vfunc=lambda x:np.abs(x)
# vfunc=lambda x:np.abs(np.real(x))
# vfunc=lambda x:np.abs(np.imag(x))

x2ylims={
    ('24','0'):([0,0.6],[0,0.5]),
    ('24','1'):([0,0.1],[0,0.1]),
    ('48','0'):([0,0.06],[0,0.15]),
    ('48','1'):([0,0.02],[0,1.]),
}

x2fitmax={}
x2fitmins={}

rela_threshold=0.2
fitminmin=7

try:
    x2fitmin
except:
    x2fitmin={}
    
x2fitcov={}

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)
    xUnit=lat.a
    fig, axs = yu.getFigAxs(4,2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [2, 1]})
    
    for i_opsL, opsL in enumerate(['0','1']):
        pt,ofs=L2ptofs[opsL]
        Nop=len(ofs)
        
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops),data['VEV']['pi0f']]
        
        def func(dat):
            t=yu.meanDeep(dat)
            t2pt,tVEVpi0f=t
            # t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
            (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
            eVecs=yu.renormalize_eVecs(eVecs)
            t=eVecs[:,0,:].T
            t=t/t[0,:]
            eVecs_inv_tin=np.linalg.inv(eVecs)
            W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
            tt=np.concatenate([[W],t])
            return vfunc(tt)
        
        mean,err,cov=yu.jackknife(dat,func,sl_key=('GEVP_results_0sub',ensL,opsL))
        
        if (ensL,opsL) not in x2fitmax:
            fitmax=len(mean[0])-1
            flag=False
            for t in range(1,fitmax+1):
                for ind in range(len(mean)):
                    rela=np.abs(err[ind][t]/mean[ind][t])
                    if rela>rela_threshold:
                        fitmax=t-1
                        flag=True
                        break
                if flag:
                    break
            x2fitmax[(ensL,opsL)]=fitmax
            x2fitmins[(ensL,opsL)]=range(fitminmin,fitmax)
            
        
        irow=2*i_opsL+0;icol=0
        for iop in range(1,Nop):
            ind=iop+1
            tmin=dtt0+1; tmax=len(mean[ind]); tmax=x2fitmax[(ensL,opsL)]+1
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            if opsL is '0':
                label=r'$i=N(1)\pi(1)$'
            else:
                label=r'$i=N(1)\pi(0)$' if iop is 1 else r'$i=N(0)\pi(1)$'
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc='white',label=label)

            tmin=dtt0+1; tmin=x2fitmax[(ensL,opsL)]+1; tmax=len(mean[ind])
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc='white')
            
            if (ensL,opsL) in x2fitmin:
                tmin=x2fitmin[(ensL,opsL)]; tmax=x2fitmax[(ensL,opsL)]+1
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop])
            
        ylim=x2ylims[(ensL,opsL)][0]
        axs[irow,icol].set_ylim(ylim)
        # axs[irow,icol].set_ylim(-ylim[1],ylim[1])
        
        irow=2*i_opsL+1;icol=0
        ind=0
        tmin=dtt0+1; tmax=len(mean[ind]);  tmax=x2fitmax[(ensL,opsL)]+1
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
        tmin=dtt0+1; tmin=x2fitmax[(ensL,opsL)]+1; tmax=len(mean[ind])
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
        if (ensL,opsL) in x2fitmin:
            tmin=x2fitmin[(ensL,opsL)]; tmax=x2fitmax[(ensL,opsL)]+1
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0])
        ylim=x2ylims[(ensL,opsL)][1]
        axs[irow,icol].set_ylim(ylim)
        # axs[irow,icol].set_ylim(-ylim[1],ylim[1])
        
        fits=[]
        fitmax=x2fitmax[(ensL,opsL)]
        for fitmin in x2fitmins[(ensL,opsL)]:
            # print(fitmin,fitmax)
            fitsizeunit=fitmax-fitmin+1
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                # t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
                eVecs=yu.renormalize_eVecs(eVecs)
                t=eVecs[:,0,:].T
                t=t/t[0,:]
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                tt=np.concatenate([[W],t[1:]])[:,fitmin:fitmax+1]
                if opsL=='0':
                    return np.real(tt)
                return np.concatenate([np.real(tt),np.imag(tt)]) 
            
            if (ensL,opsL) in x2fitmin and fitmin==x2fitmin[(ensL,opsL)]:
                t_ops=ops.copy()
                ops=yu.getops(pt,'l1',ofs)
                dat2=[get2ptMat(ops),data['VEV']['pi0f']]
                _,_,cov1=yu.jackknife(dat2,func,outputFlatten=True)
                ops=yu.getops(pt,'l2',ofs)
                dat2=[get2ptMat(ops),data['VEV']['pi0f']]
                _,_,cov2=yu.jackknife(dat2,func,outputFlatten=True)
                x2fitcov[(ensL,opsL)]=(cov1,cov2)
                ops=t_ops

            if opsL=='0':
                def fitfunc(Wr,v1r):
                    return [[Wr]*fitsizeunit, [v1r]*fitsizeunit] 
            elif opsL=='1':
                def fitfunc(Wr,v1r,v2r,Wi,v1i,v2i):
                    return [[Wr]*fitsizeunit, [v1r]*fitsizeunit, [v2r]*fitsizeunit, [Wi]*fitsizeunit, [v1i]*fitsizeunit, [v2i]*fitsizeunit]
            
            res=yu.fit(dat,func,fitfunc,sl_key=('GEVP_fit_0sub',ensL,opsL,fitmin))
            pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,_=res
            
            # print(ensL,opsL)
            # print(func(dat))
            # print(pars_mean)
            
            if opsL=='0':
                pars_mean_C=pars_mean
                pars_err_C=pars_err
            else:
                pars_mean_C=pars_mean[:Nop]+1j*pars_mean[Nop:]
                pars_err_C=pars_err[:Nop]+1j*pars_err[Nop:]
            fits.append((pars_mean_C,pars_err_C,chi2R_mean,Ndof))
            
        pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)
        imfit=np.argmax(probs)
        x2fitmin[(ensL,opsL)]=x2fitmins[(ensL,opsL)][imfit]
        
        for ifit,fitmin in enumerate(x2fitmins[(ensL,opsL)]): 
            mfc='white' if ifit!=imfit else None
            pars_mean_C,pars_err_C,chi2R_mean,_=fits[ifit]
            irow=2*i_opsL+0; icol=1
            for iop in range(1,Nop):
                tmean=vfunc(pars_mean_C[iop]); terr=vfunc(pars_err_C[iop])
                axs[irow,icol].errorbar([fitmin*xUnit],tmean,terr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc=mfc)
                ylim=x2ylims[(ensL,opsL)][0]
                shift_chi2=(ylim[1]-ylim[0])/15
                axs[irow,icol].annotate("%0.1f" %chi2R_mean,(fitmin*xUnit,tmean-terr-shift_chi2),color=['r','g','b'][iop],size=15)

            irow=2*i_opsL+1; icol=1
            iop=0
            tmean=vfunc(pars_mean_C[iop]); terr=vfunc(pars_err_C[iop])
            axs[irow,icol].errorbar([fitmin*xUnit],tmean,terr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc=mfc)
            ylim=x2ylims[(ensL,opsL)][1]
            shift_chi2=(ylim[1]-ylim[0])/15
            axs[irow,icol].annotate("%0.1f" %chi2R_mean,(fitmin*xUnit,tmean-terr-shift_chi2),color=['r','g','b'][iop],size=15)

        axs[2*i_opsL+1,1].set_xlim([0.55,1.25] if ensL is '24' else [0.55,1.4])
        
        # axs[2*i_opsL+0,0].text(.55, .95, L2T_pt[opsL], ha='center', va='top', size=30, transform=axs[2*i_opsL+0,0].transAxes)
        axs[2*i_opsL+1,0].text(.05, .95, L2T_pt[opsL], ha='left', va='top', size=30, transform=axs[2*i_opsL+1,0].transAxes)
        
        
    axs[2*1+1,0].set_xlabel(r'$t$ [fm]',fontsize=40)
    axs[2*1+1,1].set_xlabel(r'$t_{\mathrm{min}}$ [fm]',fontsize=40)
    axs[0,0].set_ylabel(r'$|v_{N,O_{i}}/v_{N,O_{N(0)}}|$',fontsize=40)
    axs[1,0].set_ylabel(r'$|W|}$',fontsize=40)
    axs[2,0].set_ylabel(r'$|v_{N,O_{i}}/v_{N,O_{N(1)}}|$',fontsize=40)
    axs[3,0].set_ylabel(r'$|W|}$',fontsize=40)
        
    
    # if ensL is '48':
    #     axs[2,0].set_yticks([0,0.006,0.012,0.018])
    
    if ensL is '48':
        axs[3,1].set_xticks([0.7,1.0,1.3])
    else:
        axs[3,1].set_xticks([0.7,1.0])
        
    axs[0,0].legend(loc='upper left')
    axs[2,0].legend(loc='upper left')
    # if ensL is '24':
    #     axs[0,0].legend(loc=(0.07,0.7))
    #     axs[2,0].legend(loc=(0.07,0.2))
    # else:
    #     axs[0,0].legend(loc=(0.07,0.7))
    #     axs[2,0].legend(loc=(0.07,0.2))
        
    for irow in range(4):
        for icol in range(2):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=30)
        
    plt.tight_layout()
    plt.savefig('fig/2ptGEVP_'+ensL+'_0sub.pdf')
    plt.close()
     
for ensL in ['48']:
    run(ensL)
    run(ensL)
    print(ensL)
        # break
    # break

print()    
print(x2fitmin)
print(x2fitmax)
x2fitmin={}

In [ ]:
dtt0=4

# yu.sl_reset()
vfunc=lambda x:np.abs(x)
# vfunc=lambda x:np.abs(np.real(x))
# vfunc=lambda x:np.abs(np.imag(x))

x2ylims={
    ('24','0'):([0,0.6],[0,0.5]),
    ('24','1'):([0,0.1],[0,0.1]),
    ('48','0'):([0,0.06],[0,0.15]),
    ('48','1'):([0,0.02],[0,0.03]),
}

x2fitmax={}
x2fitmins={}

rela_threshold=0.2
fitminmin=7

try:
    x2fitmin
except:
    x2fitmin={}
    
x2fitcov={}

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)
    xUnit=lat.a
    fig, axs = yu.getFigAxs(4,2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [2, 1]})
    
    for i_opsL, opsL in enumerate(['0','1']):
        pt,ofs=L2ptofs[opsL]
        Nop=len(ofs)
        
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops),data['VEV']['pi0f']]
        
        def func(dat):
            t=yu.meanDeep(dat)
            t2pt,tVEVpi0f=t
            t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
            (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
            eVecs=yu.renormalize_eVecs(eVecs)
            t=eVecs[:,0,:].T
            t=t/t[0,:]
            eVecs_inv_tin=np.linalg.inv(eVecs)
            W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
            tt=np.concatenate([[W],t])
            return vfunc(tt)
        
        mean,err,cov=yu.jackknife(dat,func,sl_key=('GEVP_results',ensL,opsL))
        
        if (ensL,opsL) not in x2fitmax:
            fitmax=len(mean[0])-1
            flag=False
            for t in range(1,fitmax+1):
                for ind in range(len(mean)):
                    rela=np.abs(err[ind][t]/mean[ind][t])
                    if rela>rela_threshold:
                        fitmax=t-1
                        flag=True
                        break
                if flag:
                    break
            x2fitmax[(ensL,opsL)]=fitmax
            x2fitmins[(ensL,opsL)]=range(fitminmin,fitmax)
            
        
        irow=2*i_opsL+0;icol=0
        for iop in range(1,Nop):
            ind=iop+1
            tmin=dtt0+1; tmax=len(mean[ind]); tmax=x2fitmax[(ensL,opsL)]+1
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            if opsL is '0':
                label=r'$k=N(1)\pi(1)$'
            else:
                label=r'$k=N(1)\pi(0)$' if iop is 1 else r'$k=N(0)\pi(1)$'
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc='white',label=label)

            tmin=dtt0+1; tmin=x2fitmax[(ensL,opsL)]+1; tmax=len(mean[ind])
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc='white')
            
            if (ensL,opsL) in x2fitmin:
                tmin=x2fitmin[(ensL,opsL)]; tmax=x2fitmax[(ensL,opsL)]+1
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][iop],fmt=['s','d','o'][iop])
            
        ylim=x2ylims[(ensL,opsL)][0]
        axs[irow,icol].set_ylim(ylim)
        # axs[irow,icol].set_ylim(-ylim[1],ylim[1])
        
        irow=2*i_opsL+1;icol=0
        ind=0
        tmin=dtt0+1; tmax=len(mean[ind]);  tmax=x2fitmax[(ensL,opsL)]+1
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
        tmin=dtt0+1; tmin=x2fitmax[(ensL,opsL)]+1; tmax=len(mean[ind])
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
        if (ensL,opsL) in x2fitmin:
            tmin=x2fitmin[(ensL,opsL)]; tmax=x2fitmax[(ensL,opsL)]+1
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean,tErr,color=['r','g','b'][0],fmt=['s','d','o'][0])
        ylim=x2ylims[(ensL,opsL)][1]
        axs[irow,icol].set_ylim(ylim)
        # axs[irow,icol].set_ylim(-ylim[1],ylim[1])
        
        fits=[]
        fitmax=x2fitmax[(ensL,opsL)]
        for fitmin in x2fitmins[(ensL,opsL)]:
            # print(fitmin,fitmax)
            fitsizeunit=fitmax-fitmin+1
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dtt0)
                eVecs=yu.renormalize_eVecs(eVecs)
                t=eVecs[:,0,:].T
                t=t/t[0,:]
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                tt=np.concatenate([[W],t[1:]])[:,fitmin:fitmax+1]
                if opsL=='0':
                    return np.real(tt)
                return np.concatenate([np.real(tt),np.imag(tt)]) 
            
            if (ensL,opsL) in x2fitmin and fitmin==x2fitmin[(ensL,opsL)]:
                t_ops=ops.copy()
                ops=yu.getops(pt,'l1',ofs)
                dat2=[get2ptMat(ops),data['VEV']['pi0f']]
                _,_,cov1=yu.jackknife(dat2,func,outputFlatten=True)
                ops=yu.getops(pt,'l2',ofs)
                dat2=[get2ptMat(ops),data['VEV']['pi0f']]
                _,_,cov2=yu.jackknife(dat2,func,outputFlatten=True)
                x2fitcov[(ensL,opsL)]=(cov1,cov2)
                ops=t_ops

            if opsL=='0':
                def fitfunc(Wr,v1r):
                    return [[Wr]*fitsizeunit, [v1r]*fitsizeunit] 
            elif opsL=='1':
                def fitfunc(Wr,v1r,v2r,Wi,v1i,v2i):
                    return [[Wr]*fitsizeunit, [v1r]*fitsizeunit, [v2r]*fitsizeunit, [Wi]*fitsizeunit, [v1i]*fitsizeunit, [v2i]*fitsizeunit]
            
            res=yu.fit(dat,func,fitfunc,sl_key=('GEVP_fit',ensL,opsL,fitmin))
            pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,_=res
            
            # print(ensL,opsL)
            # print(func(dat))
            # print(pars_mean)
            
            if opsL=='0':
                pars_mean_C=pars_mean
                pars_err_C=pars_err
            else:
                pars_mean_C=pars_mean[:Nop]+1j*pars_mean[Nop:]
                pars_err_C=pars_err[:Nop]+1j*pars_err[Nop:]
            fits.append((pars_mean_C,pars_err_C,chi2R_mean,Ndof))
            
        pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)
        imfit=np.argmax(probs)
        x2fitmin[(ensL,opsL)]=x2fitmins[(ensL,opsL)][imfit]
        
        for ifit,fitmin in enumerate(x2fitmins[(ensL,opsL)]): 
            mfc='white' if ifit!=imfit else None
            pars_mean_C,pars_err_C,chi2R_mean,_=fits[ifit]
            irow=2*i_opsL+0; icol=1
            for iop in range(1,Nop):
                tmean=vfunc(pars_mean_C[iop]); terr=vfunc(pars_err_C[iop])
                axs[irow,icol].errorbar([fitmin*xUnit],tmean,terr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc=mfc)
                ylim=x2ylims[(ensL,opsL)][0]
                shift_chi2=(ylim[1]-ylim[0])/15
                axs[irow,icol].annotate("%0.1f" %chi2R_mean,(fitmin*xUnit,tmean-terr-shift_chi2),color=['r','g','b'][iop],size=15)

            irow=2*i_opsL+1; icol=1
            iop=0
            tmean=vfunc(pars_mean_C[iop]); terr=vfunc(pars_err_C[iop])
            axs[irow,icol].errorbar([fitmin*xUnit],tmean,terr,color=['r','g','b'][iop],fmt=['s','d','o'][iop],mfc=mfc)
            ylim=x2ylims[(ensL,opsL)][1]
            shift_chi2=(ylim[1]-ylim[0])/15
            axs[irow,icol].annotate("%0.1f" %chi2R_mean,(fitmin*xUnit,tmean-terr-shift_chi2),color=['r','g','b'][iop],size=15)

        axs[2*i_opsL+1,1].set_xlim([0.55,1.25] if ensL is '24' else [0.55,1.4])
        
        # axs[2*i_opsL+0,0].text(.55, .95, L2T_pt[opsL], ha='center', va='top', size=30, transform=axs[2*i_opsL+0,0].transAxes)
        axs[2*i_opsL+1,0].text(.05, .95, L2T_pt[opsL], ha='left', va='top', size=30, transform=axs[2*i_opsL+1,0].transAxes)
        
        
    axs[2*1+1,0].set_xlabel(r'$t$ [fm]',fontsize=40)
    axs[2*1+1,1].set_xlabel(r'$t_{\mathrm{min}}$ [fm]',fontsize=40)
    axs[0,0].set_ylabel(r'$|v_{0,{k}}/v_{0,N}|$',fontsize=40)
    axs[1,0].set_ylabel(r'$|W|}$',fontsize=40)
    axs[2,0].set_ylabel(r'$|v_{0,{k}}/v_{0,N}|$',fontsize=40)
    axs[3,0].set_ylabel(r'$|W|}$',fontsize=40)
        
    
    # if ensL is '48':
    #     axs[2,0].set_yticks([0,0.006,0.012,0.018])
    
    if ensL is '48':
        axs[3,1].set_xticks([0.7,1.0,1.3])
    else:
        axs[3,1].set_xticks([0.7,1.0])
        
    axs[0,0].legend(loc='upper left')
    axs[2,0].legend(loc='upper left')
    # if ensL is '24':
    #     axs[0,0].legend(loc=(0.07,0.7))
    #     axs[2,0].legend(loc=(0.07,0.2))
    # else:
    #     axs[0,0].legend(loc=(0.07,0.7))
    #     axs[2,0].legend(loc=(0.07,0.2))
        
    for irow in range(4):
        for icol in range(2):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=30)
        
    plt.tight_layout()
    plt.savefig('fig/2ptGEVP_'+ensL+'.pdf')
    plt.close()
     
for ensL in ensLs:
    run(ensL)
    run(ensL)
    print(ensL)
        # break
    # break

print()    
print(x2fitmin)
print(x2fitmax)

In [ ]:
# utitlies

datmins=range(1,5+1)

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_C3pt_2st=lambda tf,tc,E0a,E0b,a00,dE1a,dE1b,ra01,ra10,ra11: a00*np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(1 + ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc)) \
    if a00!=0 else np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc))

flags['dt2pt']=False
flags['dt3pt']=True

# ylimDic={
#     'GS+_00':[6,22],'GA_00_zz':[0.5,1.6],
#     'J5_11':[-2.5,2.5],'GA_11_tz':[0.5,1.6],'GA_11_zz':[0.5,1.6],'GA_11_xx':[0.5,1.6],
#     'G5b_01':[0,25],'G5_01':[0,130],'GA_01':[0.8,1.3],'GPb_01_tz':[10,60],'GPb_01_zz':[10,60],'GP_01_tz':[10,60],'GP_01_zz':[10,60],
#     'rPCAC_1_Ptz':[0.2,1.5],'rPCAC_1_Pzz':[0.2,1.5],
# }

ylimDic={
    ('24','GS+_00'):[3,9.5], ('24','GS+_11'):[3,9.5],
    ('24','GSb+_00'):[80,220], ('24','GSb+_11'):[80,220],
    ('24','GA_00_zz'):[0.5,1.6], ('24','GA_11_tz'):[0.5,1.6], ('24','GA_11_zz'):[0.5,1.6], ('24','GA_11_xx'):[0.5,1.6],
    ('24','J5_11'):[-1.3,1.3],
    ('24','G5_01'):[0,30], ('24','GA_01'):[0.81,0.99], ('24','GP_01_tz'):[-30,40], ('24','GP_01_zz'):[5,17],
    ('24','rPCAC_1_Ptz'):[-2,3], ('24','rPCAC_1_Pzz'):[0.2,1.5],

    ('48','GS+_00'):[4,12.5], ('48','GS+_11'):[4,12.5], 
    ('48','GSb+_00'):[15,60], ('48','GSb+_11'):[15,60],
    ('48','GA_00_zz'):[0.5,1.6], ('48','GA_11_tz'):[0.5,1.6], ('48','GA_11_zz'):[0.5,1.6], ('48','GA_11_xx'):[0.5,1.6],
    ('48','J5_11'):[-1.3,1.3],
    ('48','G5_01'):[0,85], ('48','GA_01'):[0.96,1.24], ('48','GP_01_tz'):[-120,150], ('48','GP_01_zz'):[10,55],
    ('48','rPCAC_1_Ptz'):[-2,3], ('48','rPCAC_1_Pzz'):[0.2,1.5],
}

FFinPCAC=['rPCAC_1_Ptz','rPCAC_1_Pzz']

FF2simone={
    'GA_00_zz':'GA_0', 'GA_11_tz':'GA_0', 'GA_11_zz':'GA_0', 'GA_11_xx':'GA_0', 'GA_01':'GA_1',
    'GPb_01_tz':'GPb_1','GP_01_tz':'GP_1', 'GPb_01_zz':'GPb_1','GP_01_zz':'GP_1',
    'G5b_01':'G5b_1','G5_01':'G5_1',
    'rPCAC_1_Ptz':'rPCAC_1', 'rPCAC_1_Pzz':'rPCAC_1',
}

FF2opabsL={
    'GS+_00':('0','0'), 'GS+_11':('1','1'), 
    'GSb+_00':('0','0'), 'GSb+_11':('1','1'),
    'GA_00_zz':('0','0'), 'GA_11_tz':('1','1'), 'GA_11_zz':('1','1'), 'GA_11_xx':('1','1'),
    'J5_11':('1','1'), 
    'G5_01':('0','1'), 'GA_01':('0','1'), 'GP_01_tz':('0','1'), 'GP_01_zz':('0','1'),
    'rPCAC_1_Ptz':('0','1'), 'rPCAC_1_Pzz':('0','1'),
}

FF2label_ratio={
    'GS+_00':r'$\bar{R}^{\Gamma_0}_{S^{u+d}}(\vec{0},\vec{0})$', 'GS+_11':r'$\bar{R}^{\Gamma_0}_{S^{u+d}}(\vec{1}_k,\vec{1}_k)$', 
    'GSb+_00':r'$\bar{R}^{\Gamma_0}_{\sigma}(\vec{0},\vec{0})$ [MeV]', 'GSb+_11':r'$\bar{R}^{\Gamma_0}_{\sigma}(\vec{1}_z,\vec{1}_z)$ [MeV]', 
    'GA_00_zz':r'$\bar{R}^{\Gamma_z}_{A_z^{u-d}}(\vec{0},\vec{0})$',  'GA_11_tz':r'$\bar{R}^{\Gamma_z}_{A_4^{u-d}}(\vec{1}_z,\vec{1}_z)$', 'GA_11_zz':r'$\bar{R}^{\Gamma_z}_{A_z^{u-d}}(\vec{1}_z,\vec{1}_z)$', 'GA_11_xx':r'$\bar{R}^{\Gamma_{x}}_{A_{x}^{u-d}}(\vec{1}_{z},\vec{1}_{z})$',
    'J5_11':r'$\bar{R}^{\Gamma_z}_{P^{u-d}}(\vec{1}_z,\vec{1}_z)$',
    'G5_01':r'$\bar{R}^{\Gamma_z}_{P^{u-d}}(\vec{0},\vec{1}_z)$', 'GA_01':r'$\bar{R}^{\Gamma_{x}}_{A_{x}^{u-d}}(\vec{0},\vec{1}_z)$', 'GP_01_tz':r'$\bar{R}^{\Gamma_{z}}_{A_{4}^{u-d}}(\vec{0},\vec{1}_z)$', 'GP_01_zz':r'$\bar{R}^{\Gamma_{z}}_{A_{z}^{u-d}}(\vec{0},\vec{1}_z)$',
    'rPCAC_1_Ptz':r'$\bar{R}^{\Gamma_{z}}_{\mathrm{PCAC}_4^{u-d}}(\vec{0},\vec{1}_z)$', 'rPCAC_1_Pzz':r'$\bar{R}^{\Gamma_{z}}_{\mathrm{PCAC}_z^{u-d}}(\vec{0},\vec{1}_z)$', 
}

FF2label_charge={
    'GS+_00':r'$g_S^{u+d}$', 'GS+_11':r'$g_S^{u+d}$', 
    'GSb+_00':r'$\sigma_{\pi N}$ [MeV]', 'GSb+_11':r'$\sigma_{\pi N}$ [MeV]', 
    'GA_00_zz':r'$g_A^{u-d}$', 'GA_11_tz':r'$g_A^{u-d}$', 'GA_11_zz':r'$g_A^{u-d}$', 'GA_11_xx':r'$g_A^{u-d}$',
    'J5_11':r'$0$',
    'G5_01':r'$G_5^{u-d}(Q_1^2)$', 'GA_01':r'$G_A^{u-d}(Q_1^2)$', 'GP_01_tz':r'$G_P^{u-d}(Q_1^2)$', 'GP_01_zz':r'$G_P^{u-d}(Q_1^2)$',
    'rPCAC_1_Ptz':r'$r^{u-d}_{\mathrm{PCAC}}(Q_1^2)$', 'rPCAC_1_Pzz':r'$r^{u-d}_{\mathrm{PCAC}}(Q_1^2)$', 
}

simone={
    'GA_0':[1.245,0.031],
    'GA_1':[1.129,0.027],
    'GPb_1':[48.9,2.8], 'GP_1':[45.2,2.6],
    'G5b_1':[12.4,0.5], 'G5_1':[62.2,0.3],
    'rPCAC_1':[1.06,0.05],
}

gA_exp=1.27641

xlim_rainbow=[-0.6,0.6]
xticks_rainbow=[-0.4,-0.2,0,0.2,0.4]
xlabel_rainbow=r'$t_{\mathrm{ins}}-t_{\mathrm{s}}/2$ [fm]'
xlim_fit=[0,0.6]
xticks_fit=[0.1,0.3,0.5]
xlabel_fit=r'$t_{\mathrm{ins,min}}$ [fm]'

def FF2pips(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_Fpi=92.9/lat.aInv
    t_aQ2=(qk**2-(EN-mN)**2)
    t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
    t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
    t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
    
    t_GS_00=lat.ZP[0]
    t_GS_11=lat.ZP[0]/(mN/EN)
    
    t_GSb_00=lat.amu*lat.aInv
    t_GSb_11=lat.amu*lat.aInv/(mN/EN)
    
    t_A2pNN_0=mN/t_Fpi
    t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
    t_52pNN_1=(lat.amu/lat.ZP[0])*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
    t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
    
    t_GAzz_00=(-2)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA[0]; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
    t_GAtz_11=(-2)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA[0]; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
    t_GAzz_11=(-2)*(2*EN*(EN+mN))/(2j*(2*mN*(EN+mN)+2*qk**2)) * lat.ZA[0]; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
    t_GAxx_11=(-2)*(2*EN*(EN+mN))/(2j*(2*mN*(EN+mN))) * lat.ZA[0]; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
    
    t_GA_01=(-2)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA[0]
    t_G5_01=(-2*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS[0]; t_G5b_01=t_G5_01 * t_52pNN_1
    t_GP_01_tz=(-2)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA[0]; t_GP_01_tz_A=-(2*mN)/(EN-mN)
    t_GP_01_zz=(-2)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA[0]; t_GP_01_zz_A=-(2*mN)/(EN-mN)

    t_PCAC_G5=(lat.amu/lat.ZP[0])/mN
    t_PCAC_GP=t_aQ2/(4*mN**2)
    
    pips={
        'GS+_00':[(t_GS_00,'id_j+','P0')],
        'GS+_11':[(t_GS_11,'id_j+','P0')],
        'GSb+_00':[(t_GSb_00,'id_j+','P0')],
        'GSb+_11':[(t_GSb_11,'id_j+','P0')],
        
        'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
        'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
        'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
        'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
        'J5_11':[(lat.ZS[0],'g5_j-','Pz')],

        
        'G5b_01':[(t_G5b_01,'g5_j-','Pz')],'G5_01':[(t_G5_01,'g5_j-','Pz')],
        'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
        'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
        'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
        'GP_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A,'g5gy_j-','Py'),(t_GP_01_tz,'g5gt_j-','Pz')],
        'GP_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A,'g5gy_j-','Py'),(t_GP_01_zz,'g5gz_j-','Pz')],
        
        # 'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
        # 'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA[0],'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA[0],'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP[0]*lat.ZS[0],'g5_j-','Pz')],
        
        # 'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        # 'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
    
        'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
        'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        
        # 'rPCAC2_1':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),(2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
    }[FF]
    return pips

def FF2ren(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    ZSbyZPZA=yu.propagateError(lambda ZSPA:ZSPA[0]/(ZSPA[1]*ZSPA[2]),[lat.ZS[0],lat.ZP[0],lat.ZA[0]],np.diag([lat.ZS[1],lat.ZP[1],lat.ZA[1]])**2)
    ZSbyZPZA=(ZSbyZPZA[0],np.sqrt(ZSbyZPZA[1]))
    ren={
        'GS+_00':lat.ZP,'GS+_11':lat.ZP,'GSb+_00':[1,1e-8],'GSb+_11':[1,1e-8],'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
        'G5b_01':lat.ZSbyZP,'G5_01':lat.ZS,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,'GP_01_tz':lat.ZA,'GP_01_zz':lat.ZA, 'rPCAC_1_Ptz':ZSbyZPZA,'rPCAC_1_Pzz':ZSbyZPZA
    }[FF]
    return ren

def eVecs2eVecW(eVecs_ltni,ensL,opsL):
    if eVecs_ltni.shape[-1]==1:
        return (np.zeros((len(eVecs_ltni),1))+1,np.zeros((len(eVecs_ltni))+1))
    
    W_l=[]; eVec_li=[]
    eVecs_ltni=yu.renormalize_eVecs(eVecs_ltni)
    for l in range(len(eVecs_ltni)):
        eVecs=eVecs_ltni[l]
        fitsizeunit=len(eVecs)
        t=eVecs[:,0,:].T
        t=t/t[0,:]
        eVecs_inv_tin=np.linalg.inv(eVecs)
        W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
        tt=np.concatenate([[W],t[1:]])
        if opsL=='0':
            t=np.real(tt)
            def fitfunc(Wr,v1r):
                return [[Wr]*fitsizeunit, [v1r]*fitsizeunit]
            Wr,v1r=yu.fit0(fitfunc,t,x2fitcov[(ensL,opsL)][l])[0]
            W_l.append(Wr); eVec_li.append([1,v1r])
        else:
            t=np.concatenate([np.real(tt),np.imag(tt)]) 
            def fitfunc(Wr,v1r,v2r,Wi,v1i,v2i):
                return [[Wr]*fitsizeunit, [v1r]*fitsizeunit, [v2r]*fitsizeunit, [Wi]*fitsizeunit, [v1i]*fitsizeunit, [v2i]*fitsizeunit]
            Wr,v1r,v2r,Wi,v1i,v2i=yu.fit0(fitfunc,t,x2fitcov[(ensL,opsL)][l])[0]
            W_l.append(Wr+1j*Wi); eVec_li.append([1,v1r+1j*v1i,v2r+1j*v2i])
    W_l=np.array(W_l); eVec_li=np.array(eVec_li)   
    return (eVec_li,W_l)

tfs=[10,12,14]
def get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype='Id'):
    (eVeca_li,Wa_l)=eVecs2eVecW(eVecsa_ltni,ensL,opasL)
    (eVecb_li,Wb_l)=eVecs2eVecW(eVecsb_ltni,ensL,opbsL)
    
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    if Itype in ['Id','Ip']:
        dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt'] and Itype in ['Id']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

def errRen(mean,err,ren):
    tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
    tcov+=1e-8
    tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
    return tmean,np.sqrt(np.diag(tcov))

proj2mat={
    'P0':np.array([[1,0],[0,1]])/2,
    'Px':np.array([[0,1],[1,0]])/2,
    'Py':np.array([[0,-1j],[1j,0]])/2,
    'Pz':np.array([[1,0],[0,-1]])/2,
}


In [ ]:
# 2pt E-energy fits

ens2info={
    ('cA211.530.24','0,0,0'):
        {
            'tmins_1st':range(2,13+1),
            'tmins_2st':range(1,8+1),
            'tmins_3st':range(1,2+1),
            },
    ('cA2.09.48','0,0,0'):
        {
            'tmins_1st':range(7,18+1),
            'tmins_2st':range(1,8+1),
            'tmins_3st':range(1,2+1),
            },
    ('cA211.530.24','0,0,1'):
        {
            'tmins_1st':range(3,13+1),
            'tmins_2st':range(1,8+1),
            'tmins_3st':range(1,2+1),
            },
    ('cA2.09.48','0,0,1'):
        {
            'tmins_1st':range(6,18+1),
            'tmins_2st':range(1,8+1),
            'tmins_3st':range(1,2+1),
            },
}

ens2tfRange={}
ens2p0={}

def run(ensL,opsL):
    ens=L2ens[ensL]
    updateEns(ens)
    pt,ofs=L2ptofs[opsL]
    xUnit=lat.a; yUnit=lat.aInv/1000
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
    # fig.suptitle((ens,pt))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    
    tList=range(x2fitmin[(ensL,opsL)],x2fitmax[(ensL,opsL)]+1)
    
    flag_ylim=True
    for case in ['noGEVP','GEVP']:
        mfc={'noGEVP':'white','GEVP':None}[case]
        xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
        ops=yu.getops(pt,'l1',ofs)
        if case=='noGEVP':
            ops=[ops[0]]
        dat=[get2ptMat(ops),data['VEV']['pi0f']] 
        
        # get tfmax
        def func_2pt(dat):
            t2pt,tVEVpi0f=yu.meanDeep(dat)
            t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
            _,eVecs=yu.GEVP(t2pt,-dtt0,tList=tList)
            (eVec_li,W)=eVecs2eVecW(np.array([eVecs]),ensL,opsL)
            eVec=eVec_li[0]
            t=np.einsum('i,tij,j->t',np.conj(eVec),t2pt,eVec)
            return np.real(t)
        mean,err,cov=yu.jackknife(dat,func_2pt,sl_key=('run2pt',ens,pt,case,1))
        t=np.abs(err[0]/mean[0])
        tfmax=Tpack
        for tf in range(1,Tpack):
            if t[tf]>0.2:
                tfmax=tf
                print(tfmax)
                break
        
        def func(dat):
            t=func_2pt(dat)
            t=np.log(t/np.roll(t,-1))
            return t
        mean,err,cov=yu.jackknife(dat,func,sl_key=('run2pt',ens,pt,case,2))
        tmin=2; tmax=Tpack-1
        ax1.errorbar(np.arange(tmin,tmax)*xUnit+xshift,mean[0][tmin:tmax]*yUnit,yerr=err[0][tmin:tmax]*yUnit,color='orange',mfc=mfc)
        
        # initial pars
        pars0_initial=[1/yUnit, 8e-8, 1/yUnit, 2, 2/yUnit, 1]
        
        # 1st fit
        pars0=pars0_initial[:2]
        fits=[]
        temp_ax2=[]
        for tmin_fit in ens2info[(ens,pt)]['tmins_1st']:
            tfs=np.arange(tmin_fit,tfmax)
            def func(dat):
                t=func_2pt(dat)[tfs]
                return t
            def fitfunc(E0,c0):
                return func_C2pt_1st(tfs,E0,c0)
            res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('run2pt',ens,pt,case,3,tmin_fit))
            est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
            # print(tmin_fit,pars_mean)
            color={'noGEVP':'darkblue','GEVP':'blue'}[case]
            ax2.errorbar([tmin_fit*xUnit+xshift],est_mean[0]*yUnit,yerr=est_err[0]*yUnit,color=color,mfc=mfc)
            temp_ax2.append([chi2R_mean,tmin_fit,est_mean[0],est_err[0],color,'1st'])
            # ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[0]-est_err[0]-0.005)*yUnit),color='r')
            fits.append((est_mean,est_err,chi2R_mean,Ndof))
            
            # axs[irow,icol+2].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,pars_mean[8]-pars_err[8]-shift_chi2),color=color)
        
        pars0=pars0_initial[:4]
        fits=[]
        temp_ax3=[]
        def estimator(pars):
            return pars+[0,0,pars[0],0]
        for tmin_fit in ens2info[(ens,pt)]['tmins_2st']:
            # print('2st',tmin_fit)
            tfs=np.arange(tmin_fit,tmax+1)
            def func(dat):
                t=func_2pt(dat)[tfs]
                return t
            def fitfunc(E0,c0,dE1,rc1):
                return func_C2pt_2st(tfs,E0,c0,dE1,rc1)
            res=yu.fit(dat,func,fitfunc,estimator=estimator,pars0=pars0,sl_key=('run2pt',ens,pt,case,4,tmin_fit))
            est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
            # print(ens,pt,case,tmin_fit,pars0,est_mean)
            # print(tmin_fit,pars_mean[0],pars_err[0])
            color={'noGEVP':'darkred','GEVP':'r'}[case]
            ax2.errorbar([tmin_fit*xUnit+xshift],est_mean[0]*yUnit,yerr=est_err[0]*yUnit,color=color,mfc=mfc,fmt='d')
            temp_ax2.append([chi2R_mean,tmin_fit,est_mean[0],est_err[0],color,'2st'])
            # ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[0]-est_err[0]-0.005)*yUnit),color='g')
            ax3.errorbar([tmin_fit*xUnit+xshift],est_mean[2]*yUnit,yerr=est_err[2]*yUnit,color=color,mfc=mfc,fmt='d')
            temp_ax3.append([chi2R_mean,tmin_fit,est_mean[2],est_err[2],color,'2st'])
            # ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[2]-est_err[2]-0.005)*yUnit),color='g')
            fits.append((est_mean,est_err,chi2R_mean,Ndof))
        est_mean_MA,est_err_MA,probs=yu.modelAvg(fits)
        tmean,terr=est_mean_MA[0],est_err_MA[0]; 
        if flag_ylim:
            ylim1=np.array([tmean-terr*10,tmean+terr*40])*yUnit
            ylim2=np.array([tmean-terr*10,tmean+terr*20])*yUnit
        tmean,terr=est_mean_MA[2],est_err_MA[2]; 
        if flag_ylim:
            ylim3=np.array([tmean-terr*4,tmean+terr*8])*yUnit
            ax1.set_ylim(ylim1); ax2.set_ylim(ylim2); ax3.set_ylim(ylim3)
            flag_ylim=False
        ens2tfRange[(ens,pt,case)]=[ens2info[(ens,pt)]['tmins_2st'][probs.argmax()],tfmax]
        t=fits[4][0]; t[2]=t[2]-t[0]
        ens2p0[(ens,pt,case)]=t
        
        for ax in [ax2,ax3]:
            ax.axvline(x=(ens2tfRange[(ens,pt,case)][0]+ {'noGEVP':-0.2,'GEVP':-0.1}[case])*xUnit,color={'noGEVP':'darkred','GEVP':'r'}[case],linestyle={'noGEVP':'--','GEVP':'-.'}[case],marker='')
        
        # ysgn={'noGEVP':1,'GEVP':-1}[case]
        # yshift=(ylim2[1]-ylim2[0])/18
        # for chi2R_mean,tmin_fit,est_mean,est_err,color in temp_ax2:
        #     ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean+ysgn*(est_err+yshift/yUnit))*yUnit),color=color,size=18)
        # yshift=(ylim3[1]-ylim3[0])/18
        # for chi2R_mean,tmin_fit,est_mean,est_err,color in temp_ax3:
        #     ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean+ysgn*(est_err+yshift/yUnit))*yUnit),color=color,size=18)
            
        for chi2R_mean,tmin_fit,est_mean,est_err,color,nst in temp_ax2:
            corY=ylim2[0]+(ylim2[1]-ylim2[0])* ({('noGEVP','1st'):0.05,('noGEVP','2st'):0.15,('GEVP','1st'):0.9,('GEVP','2st'):0.8}[(case,nst)])
            ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,corY),color=color,size=15)
        for chi2R_mean,tmin_fit,est_mean,est_err,color,nst in temp_ax3:
            corY=ylim3[0]+(ylim3[1]-ylim3[0])* ({('noGEVP','2st'):0.1,('GEVP','2st'):0.85}[(case,nst)])
            ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,corY),color=color,size=15)
        
        # 3st fit
        # pars0=[0.44, 8e-8, 0.33, 1.75, 0.7, 1]
        # # pars0[:4]=estimatoInv(pars_mean_MA)
        # fits=[]
        # def estimator(pars):
        #     return pars+[0,0,pars[0],0,pars[0],0]
        # for tmin_fit in ens2info[(ens,pt)]['tmins_3st']:
        #     # print('3st',tmin_fit)
        #     tfs=np.arange(tmin_fit,tmax+1)
        #     def func(dat):
        #         t=func_2pt(dat)[tfs]
        #         return t
        #     def fitfunc(E0,c0,dE1,rc1,dE2,rc2):
        #         return func_C2pt_3st(tfs,E0,c0,dE1,rc1,dE2,rc2)
        #     est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof=yu.fit(dat,func,fitfunc,pars0=pars0)
        #     ax2.errorbar([tmin_fit*xUnit],est_mean[0]*yUnit,yerr=est_err[0]*yUnit,color='b',mfc=mfc)
        #     ax2.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[0]-est_err[0]-0.005)*yUnit),color='b')
        #     ax3.errorbar([tmin_fit*xUnit],est_mean[2]*yUnit,yerr=est_err[2]*yUnit,color='b',mfc=mfc)
        #     ax3.annotate("%0.1f" %chi2R_mean,(tmin_fit*xUnit,(est_mean[2]-est_err[2]-0.005)*yUnit),color='b')
        #     fits.append((est_mean,est_err,chi2R_mean,Ndof))
        
        
        # axs[2*i_opsL+0,0].text(.05, .95, L2T_pt[opsL] + r', $|\frac{v_{N,O_{i}}}{v_{N,O_{N}}}|$', ha='left', va='top', size=30, transform=axs[2*i_opsL+0,0].transAxes)
        # axs[2*i_opsL+1,0].text(.05, .95, L2T_pt[opsL] + r', $|W|$', ha='left', va='top', size=30, transform=axs[2*i_opsL+1,0].transAxes)
    
    ax1.text(0.5, .95, L2T_pt[opsL], ha='center', va='top', size=30, transform=ax1.transAxes) 
    ax3.set_xlim([0,0.85])
    ax1.set_xlabel(r'$t$ [fm]')
    ax2.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax3.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    
    ax1.set_ylabel(r'$E_N^{\mathrm{eff}}$ [GeV]')
    ax2.set_ylabel(r'$E_N$ [GeV]')
    ax3.set_ylabel(r'$E_1$ [GeV]')
    
    if (ensL,opsL) in [('24','1')]:
        ax3.set_yticks([2,4])
    
        
    plt.tight_layout()
    plt.savefig('fig/2ptFit_'+ensL+'_'+opsL+'.pdf')
    plt.close()
    print(ens,pt)

for ensL in ensLs:
    run(ensL,'0')
    run(ensL,'1')
ens2tfRange

In [ ]:
ensL='48'
ens=L2ens[ensL]
updateEns(ens)
yUnit=lat.aInv/1000

mpiC=np.array([0.06196,0.00009])*yUnit
mpi0=np.array([0.0593,0.0027])*yUnit
print(mpiC)
print(mpi0)

In [ ]:
temp={}
for ensL in ensLs:
    for opsL in ['0','1']:
        for case in ['noGEVP','GEVP']:
            ens=L2ens[ensL]
            updateEns(ens)
            pt,ofs=L2ptofs[opsL]
            xUnit=lat.a; yUnit=lat.aInv/1000; yUnit=1
            fits=[]
            for tmin_fit in ens2info[(ens,pt)]['tmins_2st']:
                res=yu.sl(('run2pt',ens,pt,case,4,tmin_fit))
                est_mean,est_err,est_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                fits.append((est_mean,est_err,chi2R_mean,Ndof))
            est_mean_MA,est_err_MA,probs=yu.modelAvg(fits)
            print(ensL,opsL,case)
            print(est_mean_MA*[yUnit,1,yUnit,1])
            print(est_err_MA*[yUnit,1,yUnit,1])
            if case is 'GEVP':
                temp[(ensL,opsL)]=(est_mean_MA*[yUnit,1,yUnit,1],est_err_MA*[yUnit,1,yUnit,1])

In [ ]:
for ensL in ['24','48']:
    updateEns(L2ens[ensL])
    mN=lat.mN; EN=lat.getaEN(1)*lat.aInv
    mN1=temp[(ensL,'0')][0][2]*lat.aInv; EN1=temp[(ensL,'1')][0][2]*lat.aInv
    EN1pi1=(lat.getaEN(1)+lat.getaEpi(1))*lat.aInv
    EN1pi0=(lat.getaEN(1)+lat.getaEpi(0))*lat.aInv
    EN0pi1=(lat.getaEN(0)+lat.getaEpi(1))*lat.aInv
    t=[ensL,mN,mN1,EN1pi1]
    print(t)
    t=[ensL,EN,EN1,EN1pi0,EN0pi1]
    print(t)
    

In [ ]:
# ratio
def run(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            # print(dat3pt[0][0][:,0,0,0,0])
            # 1/0
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                eVals_al1,_=yu.GEVP(t2pt[0],2)
                eVals_al2,_=yu.GEVP(t2pt[1],2)
                eVals_a=(eVals_al1+eVals_al2)/2
                Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                
                eVals_bl1,_=yu.GEVP(t2pt[2],2)
                eVals_bl2,_=yu.GEVP(t2pt[3],2)
                eVals_b=(eVals_bl1+eVals_bl2)/2
                Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                def t_get2(i,pip_range=None):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        if pip_range is not None and i_pip not in pip_range:
                            continue
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                
                if FF not in FFinPCAC:
                    t=[(
                        t_get2(i)
                        -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                    )/np.sqrt(
                        t2pta_t[tf]*t2ptb_t[tf]*\
                        # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                        # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                        np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                    )
                    for i,tf in enumerate(tfs)]
                else:
                    t=[(t_get2(i,[2])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)] + [(t_get2(i,[3,4,5])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)      
                                      
                # t_sum=np.array([np.sum(ele) for ele in t])
                # t_sumdif=(t_sum-np.roll(t_sum,1))/2
                # t+=[t_sumdif,t_sum]
                
                return t
            
            mean,err,_=yu.jackknife(dat,func,sl_key=('ratio',ensL,FF,case,diagsL))
            
    print(ensL,FF,end='\t') 
    
# for ensL in ensLs:
#     for FF in ['J5_11']:
#         run(ensL,FF)
#     print()

            
for ensL in ensLs:
    for FF in ['GSb+_00','GSb+_11','GA_00_zz']:
        run(ensL,FF)
    for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
        run(ensL,FF)
    for FF in ['G5_01','GA_01','GP_01_tz','GP_01_zz']:
        run(ensL,FF)
    for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
        run(ensL,FF)
    print()
    

In [ ]:
# ratio std
def run(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            # print(dat3pt[0][0][:,0,0,0,0])
            # 1/0
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                eVals_al1,_=yu.GEVP(t2pt[0],2)
                eVals_al2,_=yu.GEVP(t2pt[1],2)
                eVals_a=(eVals_al1+eVals_al2)/2
                Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                
                eVals_bl1,_=yu.GEVP(t2pt[2],2)
                eVals_bl2,_=yu.GEVP(t2pt[3],2)
                eVals_b=(eVals_bl1+eVals_bl2)/2
                Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                def t_get2(i,pip_range=None):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        if pip_range is not None and i_pip not in pip_range:
                            continue
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                
                if FF not in FFinPCAC:
                    t=[(
                        t_get2(i)
                        -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                    )/np.sqrt(
                        t2pta_t[tf]*t2ptb_t[tf]*\
                        t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                        t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                        # np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                    )
                    for i,tf in enumerate(tfs)]
                else:
                    t=[(t_get2(i,[2])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)] + [(t_get2(i,[3,4,5])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)      
                                      
                # t_sum=np.array([np.sum(ele) for ele in t])
                # t_sumdif=(t_sum-np.roll(t_sum,1))/2
                # t+=[t_sumdif,t_sum]
                
                return t
            
            mean,err,_=yu.jackknife(dat,func,sl_key=('ratio_std',ensL,FF,case,diagsL))
            
    print(ensL,FF,end='\t') 
    
# for ensL in ensLs:
#     for FF in ['J5_11']:
#         run(ensL,FF)
#     print()

            
for ensL in ensLs:
    for FF in ['GSb+_00','GSb+_11','GA_00_zz']:
        run(ensL,FF)
    for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
        run(ensL,FF)
    for FF in ['G5_01','GA_01','GP_01_tz','GP_01_zz']:
        run(ensL,FF)
    for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
        run(ensL,FF)
    print()
    

In [ ]:
# ratio I' and I''
def run(ensL,FF,Itype):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            # print(dat3pt[0][0][:,0,0,0,0])
            # 1/0
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                eVals_al1,_=yu.GEVP(t2pt[0],2)
                eVals_al2,_=yu.GEVP(t2pt[1],2)
                eVals_a=(eVals_al1+eVals_al2)/2
                Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                
                eVals_bl1,_=yu.GEVP(t2pt[2],2)
                eVals_bl2,_=yu.GEVP(t2pt[3],2)
                eVals_b=(eVals_bl1+eVals_bl2)/2
                Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype=Itype)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                def t_get2(i,pip_range=None):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        if pip_range is not None and i_pip not in pip_range:
                            continue
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                
                if FF not in FFinPCAC:
                    t=[(
                        t_get2(i)
                        -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                    )/np.sqrt(
                        t2pta_t[tf]*t2ptb_t[tf]*\
                        # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                        # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                        np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                    )
                    for i,tf in enumerate(tfs)]
                else:
                    t=[(t_get2(i,[2])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)] + [(t_get2(i,[3,4,5])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)      
                                      
                # t_sum=np.array([np.sum(ele) for ele in t])
                # t_sumdif=(t_sum-np.roll(t_sum,1))/2
                # t+=[t_sumdif,t_sum]
                
                return t
            
            mean,err,_=yu.jackknife(dat,func,sl_key=('ratio_'+Itype,ensL,FF,case,diagsL))

    print(ensL,FF,end='\t') 
    
# for ensL in ensLs:
#     for FF in ['J5_11']:
#         run(ensL,FF)
#     print()

for Itype in ['Id','Ip','Ipp']:
    for ensL in ['48']:
        for FF in ['GP_01_zz']:
            run(ensL,FF,Itype)
        for FF in ['J5_11']:
            run(ensL,FF,Itype)
        print()
        # for FF in ['GSb+_00','GSb+_11','GA_00_zz']:
        #     run(ensL,FF,Itype)
        # for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
        #     run(ensL,FF,Itype)
        # for FF in ['G5_01','GA_01','GP_01_tz','GP_01_zz']:
        #     run(ensL,FF,Itype)
        # for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
        #     run(ensL,FF,Itype)
        # print()
    

In [ ]:
# ratio I' and I''
def run(ensL,FF,Itype):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    
    for case in ['GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in ['all']:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            # print(dat3pt[0][0][:,0,0,0,0])
            # 1/0
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                eVals_al1,_=yu.GEVP(t2pt[0],2)
                eVals_al2,_=yu.GEVP(t2pt[1],2)
                eVals_a=(eVals_al1+eVals_al2)/2
                Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                
                eVals_bl1,_=yu.GEVP(t2pt[2],2)
                eVals_bl2,_=yu.GEVP(t2pt[3],2)
                eVals_b=(eVals_bl1+eVals_bl2)/2
                Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype=Itype)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                def t_get2(i,pip_range=None):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        if pip_range is not None and i_pip not in pip_range:
                            continue
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                
                if FF not in FFinPCAC:
                    t=[(
                        t_get2(i)
                        -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                    )/np.sqrt(
                        t2pta_t[tf]*t2ptb_t[tf]*\
                        # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                        # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                        np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                    )
                    for i,tf in enumerate(tfs)]
                else:
                    t=[(t_get2(i,[2])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)] + [(t_get2(i,[3,4,5])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)      
                                      
                # t_sum=np.array([np.sum(ele) for ele in t])
                # t_sumdif=(t_sum-np.roll(t_sum,1))/2
                # t+=[t_sumdif,t_sum]
                
                return t
            
            def func2(dat):
                t=func(dat)
                i_tf=-1
                tl=t[i_tf][0]; tr=t[i_tf][-1]
                t_diff=np.abs(tl-tr)
                t_ratio=np.abs(tl/tr)
                t_dr=np.abs((tl-tr)/(tl+tr))
                return [t_diff,t_ratio,t_dr]
                
            mean,err,_=yu.jackknife(dat,func2)
            print(mean)
            print(err)
            
    print(ensL,FF,end='\t') 
    
# for ensL in ensLs:
#     for FF in ['J5_11']:
#         run(ensL,FF)
#     print()

for Itype in ['Id','Ip','Ipp']:
    for ensL in ['48']:
        for FF in ['GP_01_zz']:
            run(ensL,FF,Itype)
        # for FF in ['J5_11']:
        #     run(ensL,FF,Itype)
        print()
        # for FF in ['GSb+_00','GSb+_11','GA_00_zz']:
        #     run(ensL,FF,Itype)
        # for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
        #     run(ensL,FF,Itype)
        # for FF in ['G5_01','GA_01','GP_01_tz','GP_01_zz']:
        #     run(ensL,FF,Itype)
        # for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
        #     run(ensL,FF,Itype)
        # print()
    

In [ ]:
# ratio I' and I'' _ nofit _ no subtraction

def eVecs2eVecW2(eVecs_ltni,ensL,opsL):
    if eVecs_ltni.shape[-1]==1:
        return (np.zeros((len(eVecs_ltni),1))+1,np.zeros((len(eVecs_ltni))+1))
    W_l=[]; eVec_li=[]
    eVecs_ltni=yu.renormalize_eVecs(eVecs_ltni)
    eVecs_lni=np.mean(eVecs_ltni,axis=1)
    # eVecs_lni=eVecs_ltni[:,0]
    eVecs_inv_lin=np.linalg.inv(eVecs_lni)
    
    eVec_li=eVecs_lni[:,0,:]
    W_l=1/(eVecs_lni[:,0,0]*eVecs_inv_lin[:,0,0])-1
    return (eVec_li,W_l)

def get_dt2(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype='Id'):
    (eVeca_li,Wa_l)=eVecs2eVecW2(eVecsa_ltni,ensL,opasL)
    (eVecb_li,Wb_l)=eVecs2eVecW2(eVecsb_ltni,ensL,opbsL)
    
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    if Itype in ['Id','Ip']:
        dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt'] and Itype in ['Id']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

def run(ensL,FF,Itype,subQ=True):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            # print(dat3pt[0][0][:,0,0,0,0])
            # 1/0
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                if subQ:
                    t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                    t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                    t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                    t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                    
                    for ind in range(len(t3pt)):
                        for ia,la in enumerate(['l1','l2']):
                            for ib,lb in enumerate(['l1','l2']): 
                                t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                eVals_al1,_=yu.GEVP(t2pt[0],2)
                eVals_al2,_=yu.GEVP(t2pt[1],2)
                eVals_a=(eVals_al1+eVals_al2)/2
                Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                
                eVals_bl1,_=yu.GEVP(t2pt[2],2)
                eVals_bl2,_=yu.GEVP(t2pt[3],2)
                eVals_b=(eVals_bl1+eVals_bl2)/2
                Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt2(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL,Itype=Itype)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                def t_get2(i,pip_range=None):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        if pip_range is not None and i_pip not in pip_range:
                            continue
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                
                if FF not in FFinPCAC:
                    t=[(
                        t_get2(i)
                        -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                    )/np.sqrt(
                        t2pta_t[tf]*t2ptb_t[tf]*\
                        # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                        # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                        np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                    )
                    for i,tf in enumerate(tfs)]
                else:
                    t=[(t_get2(i,[2])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)] + [(t_get2(i,[3,4,5])/t_get2(i,[0,1])) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)      
                                      
                # t_sum=np.array([np.sum(ele) for ele in t])
                # t_sumdif=(t_sum-np.roll(t_sum,1))/2
                # t+=[t_sumdif,t_sum]
                
                return t
            
            mean,err,_=yu.jackknife(dat,func,sl_key=('ratio_0fit_'+Itype,subQ,ensL,FF,case,diagsL))
            
    print(ensL,FF,end='\t') 
    
# for ensL in ensLs:
#     for FF in ['J5_11']:
#         run(ensL,FF)
#     print()

for Itype in ['Id','Ip','Ipp']:
    for subQ in [True,False]:
        for ensL in ['48']:
            for FF in ['GP_01_zz']:
                run(ensL,FF,Itype,subQ)
            for FF in ['J5_11']:
                run(ensL,FF,Itype,subQ)
            print()

In [ ]:
# separate fit
def run(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    ren=FF2ren(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func_base(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                                      
                def t_get2(i):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                t=[t2pta_t,t2ptb_t]+[ t_get2(i)-(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)
                return t
            
            if pta==ptb:
                pars0=np.concatenate([ens2p0[(ens,pta,case)],[np.mean(ylimDic[(ensL,FF)]),1]])
                fits=[]
                for datmin in datmins:
                    datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                    tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                    tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                    tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                    tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf//2+1) for tf in tfs_3pt_fit}
                    Ny_2pt=len(tfs_2pta_fit)+len(tfs_2ptb_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                    Ny=Ny_2pt+Ny_3pt
                    def func(dat):
                        t=func_base(dat)
                        t[0]=t[0][tfs_2pta_fit]
                        for i,tf in enumerate(tfs):
                            t[i+2]=t[i+2][tcs_3pt_fit[tf]-1]
                        t=[t[0]]+t[2:]
                        return t
                    def fitfunc(E0a,c0a,dE1a,rc1a,g,ra01):
                        t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                        t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0a,g*np.sqrt(c0a*c0a),dE1a,dE1a,ra01,ra01,0) for tf in tfs_3pt_fit]
                        t=[t_2pta]+t_3pt
                        return t
                    res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('fitSep',ensL,FF,case,diagsL,datmin))
            else:                                      
                pars0=np.concatenate([ens2p0[(ens,pta,case)],ens2p0[(ens,ptb,case)],[np.mean(ylimDic[(ensL,FF)]),1,1]])
                fits=[]
                for datmin in datmins:
                    datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                    tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                    tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                    tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                    tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                    Ny_2pt=len(tfs_2pta_fit)+len(tfs_2ptb_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                    Ny=Ny_2pt+Ny_3pt
                    def func(dat):
                        t=func_base(dat)
                        t[0]=t[0][tfs_2pta_fit]
                        t[1]=t[1][tfs_2ptb_fit]
                        for i,tf in enumerate(tfs):
                            t[i+2]=t[i+2][tcs_3pt_fit[tf]-1]
                        return t
                    def fitfunc(E0a,c0a,dE1a,rc1a,E0b,c0b,dE1b,rc1b,g,ra01,ra10):
                        t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                        t_2ptb=func_C2pt_2st(tfs_2ptb_fit,E0b,c0b,dE1b,rc1b)
                        t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0b,g*np.sqrt(c0a*c0b),dE1a,dE1b,ra01,ra10,0) for tf in tfs_3pt_fit]
                        t=[t_2pta,t_2ptb]+t_3pt
                        return t
                    res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('fitSep',ensL,FF,case,diagsL,datmin))
    
    print(ensL,FF,end='\t') 
    
for ensL in ensLs:
    for FF in ['GSb+_00','GSb+_11','GA_00_zz']:
        run(ensL,FF)
    for FF in ['GA_11_tz','GA_11_zz','GA_11_xx']:
        run(ensL,FF)
    for FF in ['G5_01','GA_01','GP_01_tz','GP_01_zz']:
        run(ensL,FF)
    print()
    

In [ ]:
# separate fit (with a11)
def run(ensL,FF):
    ens=L2ens[ensL]
    updateEns(ens)
    
    opasL,opbsL=FF2opabsL[FF]
    ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
    pta,ofas=ptofas; ptb,ofbs=ptofbs

    tLista=range(x2fitmin[(ensL,opasL)],x2fitmax[(ensL,opasL)]+1)
    tListb=range(x2fitmin[(ensL,opbsL)],x2fitmax[(ensL,opbsL)]+1)

    pips=FF2pips(ensL,FF)
    ren=FF2ren(ensL,FF)
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            needsVEV = pta==ptb and 'N-j' in diags and FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11','J5_11']
    
            def t_get(tf,t_insert):
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
            dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                    get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00','GS+_11','GSb+_00','GSb+_11'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f] 
            
            AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
            def func_base(dat):
                t=yu.meanDeep(dat)
                t3pt,t2pt,tVEVj,tVEVpi0f=t
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                
                for ind in range(len(t3pt)):
                    for ia,la in enumerate(['l1','l2']):
                        for ib,lb in enumerate(['l1','l2']): 
                            t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                _,eVecs_al1=yu.GEVP(t2pt[0],-dtt0,tList=tLista)
                _,eVecs_al2=yu.GEVP(t2pt[1],-dtt0,tList=tLista)
                _,eVecs_bl1=yu.GEVP(t2pt[2],-dtt0,tList=tListb)
                _,eVecs_bl2=yu.GEVP(t2pt[3],-dtt0,tList=tListb)

                eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni,ensL,opasL,opbsL)
                
                t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                
                t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                                      
                def t_get2(i):
                    res=0
                    for i_pip,pip in enumerate(pips):
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                t=[t2pta_t,t2ptb_t]+[ t_get2(i)-(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)
                return t
            
            if pta==ptb:
                pars0=np.concatenate([ens2p0[(ens,pta,case)],[np.mean(ylimDic[(ensL,FF)]),1,1]])
                fits=[]
                for datmin in datmins:
                    datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                    tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                    tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                    tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                    tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf//2+1) for tf in tfs_3pt_fit}
                    Ny_2pt=len(tfs_2pta_fit)+len(tfs_2ptb_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                    Ny=Ny_2pt+Ny_3pt
                    def func(dat):
                        t=func_base(dat)
                        t[0]=t[0][tfs_2pta_fit]
                        for i,tf in enumerate(tfs):
                            t[i+2]=t[i+2][tcs_3pt_fit[tf]-1]
                        t=[t[0]]+t[2:]
                        return t
                    def fitfunc(E0a,c0a,dE1a,rc1a,g,ra01,ra11):
                        t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                        t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0a,g*np.sqrt(c0a*c0a),dE1a,dE1a,ra01,ra01,ra11) for tf in tfs_3pt_fit]
                        t=[t_2pta]+t_3pt
                        return t
                    res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('fitSep_a11',ensL,FF,case,diagsL,datmin))
            else:                                      
                pars0=np.concatenate([ens2p0[(ens,pta,case)],ens2p0[(ens,ptb,case)],[np.mean(ylimDic[(ensL,FF)]),1,1,1]])
                fits=[]
                for datmin in datmins:
                    datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                    tfs_2pta_fit=np.arange(ens2tfRange[(ens,pta,case)][0],ens2tfRange[(ens,pta,case)][1])
                    tfs_2ptb_fit=np.arange(ens2tfRange[(ens,ptb,case)][0],ens2tfRange[(ens,ptb,case)][1])
                    tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                    tcs_3pt_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                    Ny_2pt=len(tfs_2pta_fit)+len(tfs_2ptb_fit); Ny_3pt=len([1 for tf in tfs_3pt_fit for tc in tcs_3pt_fit[tf]])
                    Ny=Ny_2pt+Ny_3pt
                    def func(dat):
                        t=func_base(dat)
                        t[0]=t[0][tfs_2pta_fit]
                        t[1]=t[1][tfs_2ptb_fit]
                        for i,tf in enumerate(tfs):
                            t[i+2]=t[i+2][tcs_3pt_fit[tf]-1]
                        return t
                    def fitfunc(E0a,c0a,dE1a,rc1a,E0b,c0b,dE1b,rc1b,g,ra01,ra10,ra11):
                        t_2pta=func_C2pt_2st(tfs_2pta_fit,E0a,c0a,dE1a,rc1a)
                        t_2ptb=func_C2pt_2st(tfs_2ptb_fit,E0b,c0b,dE1b,rc1b)
                        t_3pt=[func_C3pt_2st(tf,tcs_3pt_fit[tf],E0a,E0b,g*np.sqrt(c0a*c0b),dE1a,dE1b,ra01,ra10,ra11) for tf in tfs_3pt_fit]
                        t=[t_2pta,t_2ptb]+t_3pt
                        return t
                    res=yu.fit(dat,func,fitfunc,pars0=pars0,sl_key=('fitSep_a11',ensL,FF,case,diagsL,datmin))
    
    print(ensL,FF,end='\t') 
    
for ensL in ['48']:
    for FF in ['GA_00_zz']:
        run(ensL,FF)
        
    # for FF in ['GSb+_00','GSb+_11','GA_00_zz']:
    #     run(ensL,FF)
    # for FF in ['GA_11_tz','GA_11_zz','GA_11_xx']:
    #     run(ensL,FF)
    # for FF in ['G5_01','GA_01','GP_01_tz','GP_01_zz']:
    #     run(ensL,FF)
    print()
    

In [ ]:
# fitA5    

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)
    opsL0='0'; opsL1='1'
    ptofs0=L2ptofs[opsL0]
    ptofs1=L2ptofs[opsL1]
    
    xUnit=lat.a
   
    
    tList0=range(x2fitmin[(ensL,opsL0)],x2fitmax[(ensL,opsL0)]+1)
    tList1=range(x2fitmin[(ensL,opsL1)],x2fitmax[(ensL,opsL1)]+1)
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_Fpi=92.9/lat.aInv
    t_aQ2=(qk**2-(EN-mN)**2)
    
    FFs=['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx','J5_11']
    
    for case in ['noGEVP','GEVP']:
        pt0,ofs0=ptofs0; pt1,ofs1=ptofs1
        if case=='noGEVP':
            ofs0=[ofs0[0]]; ofs1=[ofs1[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            def t_get(tf,t_insert,opasL,opbsL):
                pta,ofas={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opasL]
                ptb,ofbs={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opbsL]
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[[t_get(tf,insert,FF2opabsL[FF][0],FF2opabsL[FF][1]) for prefactor,insert,proj in FF2pips(ensL,FF) for tf in tfs] for FF in FFs]
            dat2pt=[get2ptMat(yu.getops(pt0,'l1',ofs0),yu.diags_all),get2ptMat(yu.getops(pt0,'l2',ofs0),diags),
                    get2ptMat(yu.getops(pt1,'l1',ofs1),yu.diags_all),get2ptMat(yu.getops(pt1,'l2',ofs1),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if 0 in ['GS+_00'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,[dat2pt,datVEVj,datVEVpi0f]] 
            
            
            AInv0=yu.getCoeMat(pt0); AInv1=yu.getCoeMat(pt1)
    
            def func_base(dat):
                t=yu.meanDeep(dat)
                t3pt,temp=t
                t2pt,tVEVj,tVEVpi0f=temp
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pt0,'l1',ofs0),yu.getops(pt0,'l1',ofs0))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pt0,'l2',ofs0),yu.getops(pt0,'l2',ofs0))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(pt1,'l1',ofs1),yu.getops(pt1,'l1',ofs1))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(pt1,'l2',ofs1),yu.getops(pt1,'l2',ofs1))
                
                for ind in range(len(t3pt)):
                    FF=FFs[ind]
                    opasL,opbsL=FF2opabsL[FF]
                    pta,ofas={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opasL]
                    ptb,ofbs={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opbsL]
                    for ind2 in range(len(t3pt[ind])):
                        for ia,la in enumerate(['l1','l2']):
                            for ib,lb in enumerate(['l1','l2']): 
                                t3pt[ind][ind2][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][ind2][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                _,eVecs_0l1=yu.GEVP(t2pt[0],-dtt0,tList=tList0)
                _,eVecs_0l2=yu.GEVP(t2pt[1],-dtt0,tList=tList0)
                _,eVecs_1l1=yu.GEVP(t2pt[2],-dtt0,tList=tList1)
                _,eVecs_1l2=yu.GEVP(t2pt[3],-dtt0,tList=tList1)

                eVecs0_ltni=np.array([eVecs_0l1,eVecs_0l2])
                eVecs1_ltni=np.array([eVecs_1l1,eVecs_1l2])
                if case=='noGEVP':
                    dt2pt0_lij,dt2pt1_lij,dt3pt00_ijlm,dt3pt11_ijlm,dt3pt01_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1,np.zeros((1,1,2,2))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pt0_lij,dt2pt1_lij,dt3pt01_ijlm=get_dt(eVecs0_ltni,eVecs1_ltni,ensL,opsL0,opsL1)
                    _,_,dt3pt00_ijlm=get_dt(eVecs0_ltni,eVecs0_ltni,ensL,opsL0,opsL0)
                    _,_,dt3pt11_ijlm=get_dt(eVecs1_ltni,eVecs1_ltni,ensL,opsL1,opsL1)
                
                t2pt0_ltij=np.array([t2pt[0],t2pt[1]])
                t2pt0_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pt0_ltij,dt2pt0_lij)),axis=0)
                
                t2pt1_ltij=np.array([t2pt[2],t2pt[3]])
                t2pt1_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pt1_ltij,dt2pt1_lij)),axis=0)

                def t_get2(i,i_FF,FF):
                    opasL,opbsL=FF2opabsL[FF]
                    AInv={'0':AInv0,'1':AInv1}[opasL]; BInv={'0':AInv0,'1':AInv1}[opbsL]
                    dt3pt_ijlm={'00':dt3pt00_ijlm,'11':dt3pt11_ijlm,'01':dt3pt01_ijlm}[opasL+opbsL]
                    res=0
                    for i_pip,pip in enumerate(FF2pips(ensL,FF)):
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_FF][i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                t=[t2pt0_t,t2pt1_t]+[ t_get2(i,i_FF,FF) for i_FF,FF in enumerate(FFs) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)
                return t
                                      
            datmin=3
            pars0=np.concatenate([ens2p0[(ens,pt0,case)],ens2p0[(ens,pt1,case)],\
                yu.sl(('fitSep',ensL,'GA_00_zz',case,diagsL,datmin))[0][[4]],\
                    yu.sl(('fitSep',ensL,'GA_00_zz',case,diagsL,datmin))[0][[5]],yu.sl(('fitSep',ensL,'GA_11_tz',case,diagsL,datmin))[0][[5]],\
                        yu.sl(('fitSep',ensL,'GA_11_zz',case,diagsL,datmin))[0][[5]],yu.sl(('fitSep',ensL,'GA_11_xx',case,diagsL,datmin))[0][[5]],\
                            [1]\
                ])
            # pars0=np.concatenate([ens2p0[(ens,pt0,case)],ens2p0[(ens,pt1,case)],\
            #     [np.mean(ylimDic[(ensL,'GA_00_zz')]),1,1,1,1,1]])
            
            for datmin in datmins:
                print(ensL,diagsL,case,datmin,end='                              \r')
                datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                tfs_2pt0_fit=np.arange(ens2tfRange[(ens,pt0,case)][0],ens2tfRange[(ens,pt0,case)][1])
                tfs_2pt1_fit=np.arange(ens2tfRange[(ens,pt1,case)][0],ens2tfRange[(ens,pt1,case)][1])
                
                tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                tcs_3pt00_fit={tf:np.arange(datmin_3pt_tci,tf//2+1) for tf in tfs_3pt_fit}
                tcs_3ptJ5_fit={tf:np.arange(datmin_3pt_tci,tf//2) for tf in tfs_3pt_fit}
                tcs_3pt01_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                
                def FF2tcs_3pt_fit(FF):
                    opasL,opbsL=FF2opabsL[FF]
                    tcs_3pt_fit={'00':tcs_3pt00_fit,'11':tcs_3pt00_fit,'01':tcs_3pt01_fit}[opasL+opbsL]
                    if FF is 'J5_11':
                        tcs_3pt_fit=tcs_3ptJ5_fit
                    return tcs_3pt_fit
                
                def func(dat):
                    t=func_base(dat)
                    t[0]=t[0][tfs_2pt0_fit]
                    t[1]=t[1][tfs_2pt1_fit]
                    for i_pips in range(len(FFs)):
                        FF=FFs[i_pips]
                        tcs_3pt_fit=FF2tcs_3pt_fit(FF)
                        for i,tf in enumerate(tfs):
                            t[i_pips*len(tfs)+i+2]=t[i_pips*len(tfs)+i+2][tcs_3pt_fit[tf]-1]
                    return t
                def fitfunc(E0_0,c0_0,dE1_0,rc1_0, E0_1,c0_1,dE1_1,rc1_1, g_gA, ra01_GA_00_zz,ra01_GA_11_tz,ra01_GA_11_zz,ra01_GA_11_xx, ra01_J5_11):
                    t_2pt0=func_C2pt_2st(tfs_2pt0_fit,E0_0,c0_0,dE1_0,rc1_0)
                    t_2pt1=func_C2pt_2st(tfs_2pt1_fit,E0_1,c0_1,dE1_1,rc1_1)
                    t_3pt=[func_C3pt_2st(tf,FF2tcs_3pt_fit(FF)[tf],E0a,E0b,gcc,dE1a,dE1b,ra01,ra10,0) for FF, E0a,dE1a, E0b,dE1b, gcc,ra01,ra10 in \
                        [('GA_00_zz', E0_0,dE1_0, E0_0,dE1_0, g_gA*np.sqrt(c0_0*c0_0), ra01_GA_00_zz,ra01_GA_00_zz), 
                         ('GA_11_tz', E0_1,dE1_1, E0_1,dE1_1, g_gA*np.sqrt(c0_1*c0_1), ra01_GA_11_tz,ra01_GA_11_tz), 
                         ('GA_11_zz', E0_1,dE1_1, E0_1,dE1_1, g_gA*np.sqrt(c0_1*c0_1), ra01_GA_11_zz,ra01_GA_11_zz), 
                         ('GA_11_xx', E0_1,dE1_1, E0_1,dE1_1, g_gA*np.sqrt(c0_1*c0_1), ra01_GA_11_xx,ra01_GA_11_xx), 
                         ('J5_11', E0_1,dE1_1, E0_1,dE1_1, 0, ra01_J5_11,-ra01_J5_11),
                         ] for tf in tfs_3pt_fit]
                        
                    t=[t_2pt0,t_2pt1]+t_3pt
                    return t
                def estimator(pars): 
                    E0_0,c0_0,dE1_0,rc1_0, E0_1,c0_1,dE1_1,rc1_1, g_gA, ra01_GA_00_zz,ra01_GA_11_tz,ra01_GA_11_zz,ra01_GA_11_xx, ra01_J5_11=pars
                    return [g_gA,E0_0,E0_1]
                res=yu.fit(dat,func,fitfunc,estimator=estimator,pars0=pars0,sl_key=('fitA5',ensL,case,diagsL,datmin))

for ensL in ensLs:    
    run(ensL)

In [ ]:
# fitPCAC01

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)
    opsL0='0'; opsL1='1'
    ptofs0=L2ptofs[opsL0]
    ptofs1=L2ptofs[opsL1]
    
    xUnit=lat.a
   
    
    tList0=range(x2fitmin[(ensL,opsL0)],x2fitmax[(ensL,opsL0)]+1)
    tList1=range(x2fitmin[(ensL,opsL1)],x2fitmax[(ensL,opsL1)]+1)
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_Fpi=92.9/lat.aInv
    t_aQ2=(qk**2-(EN-mN)**2)
    
    FFs=['G5_01','GA_01','GP_01_tz','GP_01_zz']
    
    for case in ['noGEVP','GEVP']:
        pt0,ofs0=ptofs0; pt1,ofs1=ptofs1
        if case=='noGEVP':
            ofs0=[ofs0[0]]; ofs1=[ofs1[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            def t_get(tf,t_insert,opasL,opbsL):
                pta,ofas={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opasL]
                ptb,ofbs={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opbsL]
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[[t_get(tf,insert,FF2opabsL[FF][0],FF2opabsL[FF][1]) for prefactor,insert,proj in FF2pips(ensL,FF) for tf in tfs] for FF in FFs]
            dat2pt=[get2ptMat(yu.getops(pt0,'l1',ofs0),yu.diags_all),get2ptMat(yu.getops(pt0,'l2',ofs0),diags),
                    get2ptMat(yu.getops(pt1,'l1',ofs1),yu.diags_all),get2ptMat(yu.getops(pt1,'l2',ofs1),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if None in ['GS+_00'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,[dat2pt,datVEVj,datVEVpi0f]] 
            
            
            AInv0=yu.getCoeMat(pt0); AInv1=yu.getCoeMat(pt1)
    
            def func_base(dat):
                t=yu.meanDeep(dat)
                t3pt,temp=t
                t2pt,tVEVj,tVEVpi0f=temp
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pt0,'l1',ofs0),yu.getops(pt0,'l1',ofs0))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pt0,'l2',ofs0),yu.getops(pt0,'l2',ofs0))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(pt1,'l1',ofs1),yu.getops(pt1,'l1',ofs1))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(pt1,'l2',ofs1),yu.getops(pt1,'l2',ofs1))
                
                for ind in range(len(t3pt)):
                    FF=FFs[ind]
                    opasL,opbsL=FF2opabsL[FF]
                    pta,ofas={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opasL]
                    ptb,ofbs={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opbsL]
                    for ind2 in range(len(t3pt[ind])):
                        for ia,la in enumerate(['l1','l2']):
                            for ib,lb in enumerate(['l1','l2']): 
                                t3pt[ind][ind2][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][ind2][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                _,eVecs_0l1=yu.GEVP(t2pt[0],-dtt0,tList=tList0)
                _,eVecs_0l2=yu.GEVP(t2pt[1],-dtt0,tList=tList0)
                _,eVecs_1l1=yu.GEVP(t2pt[2],-dtt0,tList=tList1)
                _,eVecs_1l2=yu.GEVP(t2pt[3],-dtt0,tList=tList1)

                eVecs0_ltni=np.array([eVecs_0l1,eVecs_0l2])
                eVecs1_ltni=np.array([eVecs_1l1,eVecs_1l2])
                if case=='noGEVP':
                    dt2pt0_lij,dt2pt1_lij,dt3pt00_ijlm,dt3pt11_ijlm,dt3pt01_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1,np.zeros((1,1,2,2))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pt0_lij,dt2pt1_lij,dt3pt01_ijlm=get_dt(eVecs0_ltni,eVecs1_ltni,ensL,opsL0,opsL1)
                    _,_,dt3pt00_ijlm=get_dt(eVecs0_ltni,eVecs0_ltni,ensL,opsL0,opsL0)
                    _,_,dt3pt11_ijlm=get_dt(eVecs1_ltni,eVecs1_ltni,ensL,opsL1,opsL1)
                
                t2pt0_ltij=np.array([t2pt[0],t2pt[1]])
                t2pt0_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pt0_ltij,dt2pt0_lij)),axis=0)
                
                t2pt1_ltij=np.array([t2pt[2],t2pt[3]])
                t2pt1_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pt1_ltij,dt2pt1_lij)),axis=0)

                def t_get2(i,i_FF,FF):
                    opasL,opbsL=FF2opabsL[FF]
                    AInv={'0':AInv0,'1':AInv1}[opasL]; BInv={'0':AInv0,'1':AInv1}[opbsL]
                    dt3pt_ijlm={'00':dt3pt00_ijlm,'11':dt3pt11_ijlm,'01':dt3pt01_ijlm}[opasL+opbsL]
                    res=0
                    for i_pip,pip in enumerate(FF2pips(ensL,FF)):
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_FF][i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                t=[t2pt0_t,t2pt1_t]+[ t_get2(i,i_FF,FF) for i_FF,FF in enumerate(FFs) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)
                return t
                                      
            datmin=3
            pars0=np.concatenate([ens2p0[(ens,pt0,case)],ens2p0[(ens,pt1,case)],\
                yu.sl(('fitSep',ensL,'G5_01',case,diagsL,datmin))[0][[8,9,10]], yu.sl(('fitSep',ensL,'GA_01',case,diagsL,datmin))[0][[8,9,10]], \
                    yu.sl(('fitSep',ensL,'GP_01_tz',case,diagsL,datmin))[0][[8,9,10]], yu.sl(('fitSep',ensL,'GP_01_zz',case,diagsL,datmin))[0][[9,10]]
                ])
            
            for datmin in datmins:
                print(ensL,diagsL,case,datmin,end='                              \r')
                datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                tfs_2pt0_fit=np.arange(ens2tfRange[(ens,pt0,case)][0],ens2tfRange[(ens,pt0,case)][1])
                tfs_2pt1_fit=np.arange(ens2tfRange[(ens,pt1,case)][0],ens2tfRange[(ens,pt1,case)][1])
                
                tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                tcs_3pt00_fit={tf:np.arange(datmin_3pt_tci,tf//2+1) for tf in tfs_3pt_fit}
                tcs_3ptJ5_fit={tf:np.arange(datmin_3pt_tci,tf//2) for tf in tfs_3pt_fit}
                tcs_3pt01_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                
                def FF2tcs_3pt_fit(FF):
                    opasL,opbsL=FF2opabsL[FF]
                    tcs_3pt_fit={'00':tcs_3pt00_fit,'11':tcs_3pt00_fit,'01':tcs_3pt01_fit}[opasL+opbsL]
                    if FF is 'J5_11':
                        tcs_3pt_fit=tcs_3ptJ5_fit
                    return tcs_3pt_fit
                
                def func(dat):
                    t=func_base(dat)
                    t[0]=t[0][tfs_2pt0_fit]
                    t[1]=t[1][tfs_2pt1_fit]
                    for i_pips in range(len(FFs)):
                        FF=FFs[i_pips]
                        tcs_3pt_fit=FF2tcs_3pt_fit(FF)
                        for i,tf in enumerate(tfs):
                            t[i_pips*len(tfs)+i+2]=t[i_pips*len(tfs)+i+2][tcs_3pt_fit[tf]-1]
                    return t
                def fitfunc(E0_0,c0_0,dE1_0,rc1_0, E0_1,c0_1,dE1_1,rc1_1, g_G5,ra01_G5_01,ra10_G5_01, g_GA,ra01_GA_01,ra10_GA_01, g_GP,ra01_GP_01_tz,ra10_GP_01_tz, ra01_GP_01_zz,ra10_GP_01_zz):
                    t_2pt0=func_C2pt_2st(tfs_2pt0_fit,E0_0,c0_0,dE1_0,rc1_0)
                    t_2pt1=func_C2pt_2st(tfs_2pt1_fit,E0_1,c0_1,dE1_1,rc1_1)
                    t_3pt=[func_C3pt_2st(tf,FF2tcs_3pt_fit(FF)[tf],E0a,E0b,gcc,dE1a,dE1b,ra01,ra10,0) for FF, E0a,dE1a, E0b,dE1b, gcc,ra01,ra10 in \
                        [('G5_01', E0_0,dE1_0, E0_1,dE1_1, g_G5*np.sqrt(c0_0*c0_1), ra01_G5_01,ra10_G5_01), 
                         ('GA_01', E0_0,dE1_0, E0_1,dE1_1, g_GA*np.sqrt(c0_0*c0_1), ra01_GA_01,ra10_GA_01), 
                         ('GP_01_tz', E0_0,dE1_0, E0_1,dE1_1, g_GP*np.sqrt(c0_0*c0_1), ra01_GP_01_tz,ra10_GP_01_tz), 
                         ('GP_01_zz', E0_0,dE1_0, E0_1,dE1_1, g_GP*np.sqrt(c0_0*c0_1), ra01_GP_01_zz,ra10_GP_01_zz), 
                         ] for tf in tfs_3pt_fit]
                    t=[t_2pt0,t_2pt1]+t_3pt
                    return t
                def estimator(pars): 
                    E0_0,c0_0,dE1_0,rc1_0, E0_1,c0_1,dE1_1,rc1_1, g_G5,ra01_G5_01,ra10_G5_01, g_GA,ra01_GA_01,ra10_GA_01, g_GP,ra01_GP_01_tz,ra10_GP_01_tz, ra01_GP_01_zz,ra10_GP_01_zz=pars
                    rPCAC_5A=(lat.amu/lat.ZP[0])/lat.amN * g_G5/g_GA; rPCAC_PA=t_aQ2/(4*mN**2) * g_GP/g_GA
                    return [rPCAC_5A,rPCAC_PA,g_G5,g_GA,g_GP,E0_0,E0_1]
                res=yu.fit(dat,func,fitfunc,estimator=estimator,pars0=pars0,sl_key=('fitPCAC01',ensL,case,diagsL,datmin))

for ensL in ensLs:    
    run(ensL)

In [ ]:
# fitPCAC

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)
    opsL0='0'; opsL1='1'
    ptofs0=L2ptofs[opsL0]
    ptofs1=L2ptofs[opsL1]
    
    xUnit=lat.a
   
    
    tList0=range(x2fitmin[(ensL,opsL0)],x2fitmax[(ensL,opsL0)]+1)
    tList1=range(x2fitmin[(ensL,opsL1)],x2fitmax[(ensL,opsL1)]+1)
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_Fpi=92.9/lat.aInv
    t_aQ2=(qk**2-(EN-mN)**2)
    
    FFs=['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx','J5_11']+['G5_01','GA_01','GP_01_tz','GP_01_zz']
    
    for case in ['noGEVP','GEVP']:
        pt0,ofs0=ptofs0; pt1,ofs1=ptofs1
        if case=='noGEVP':
            ofs0=[ofs0[0]]; ofs1=[ofs1[0]]
        for diagsL in diagsLs:
            diags=L2diags[diagsL]
            def t_get(tf,t_insert,opasL,opbsL):
                pta,ofas={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opasL]
                ptb,ofbs={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opbsL]
                t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                    for lb in ['l1','l2']] for la in ['l1','l2']]
                return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
            
            dat3pt=[[t_get(tf,insert,FF2opabsL[FF][0],FF2opabsL[FF][1]) for prefactor,insert,proj in FF2pips(ensL,FF) for tf in tfs] for FF in FFs]
            dat2pt=[get2ptMat(yu.getops(pt0,'l1',ofs0),yu.diags_all),get2ptMat(yu.getops(pt0,'l2',ofs0),diags),
                    get2ptMat(yu.getops(pt1,'l1',ofs1),yu.diags_all),get2ptMat(yu.getops(pt1,'l2',ofs1),diags),]
            datVEVj=[data['VEV']['j']['id_j+' if None in ['GS+_00'] else 'g5_j-']]
            datVEVpi0f=[data['VEV']['pi0f']]
            dat=[dat3pt,[dat2pt,datVEVj,datVEVpi0f]] 
            
            
            AInv0=yu.getCoeMat(pt0); AInv1=yu.getCoeMat(pt1)
    
            def func_base(dat):
                t=yu.meanDeep(dat)
                t3pt,temp=t
                t2pt,tVEVj,tVEVpi0f=temp
                
                t2pt[0]=remove_pi0VEV(t2pt[0],tVEVpi0f[0],yu.getops(pt0,'l1',ofs0),yu.getops(pt0,'l1',ofs0))
                t2pt[1]=remove_pi0VEV(t2pt[1],tVEVpi0f[0],yu.getops(pt0,'l2',ofs0),yu.getops(pt0,'l2',ofs0))
                t2pt[2]=remove_pi0VEV(t2pt[2],tVEVpi0f[0],yu.getops(pt1,'l1',ofs1),yu.getops(pt1,'l1',ofs1))
                t2pt[3]=remove_pi0VEV(t2pt[3],tVEVpi0f[0],yu.getops(pt1,'l2',ofs1),yu.getops(pt1,'l2',ofs1))
                
                for ind in range(len(t3pt)):
                    FF=FFs[ind]
                    opasL,opbsL=FF2opabsL[FF]
                    pta,ofas={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opasL]
                    ptb,ofbs={'0':(pt0,ofs0),'1':(pt1,ofs1)}[opbsL]
                    for ind2 in range(len(t3pt[ind])):
                        for ia,la in enumerate(['l1','l2']):
                            for ib,lb in enumerate(['l1','l2']): 
                                t3pt[ind][ind2][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][ind2][:,:,:,ia,ib],tVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                
                _,eVecs_0l1=yu.GEVP(t2pt[0],-dtt0,tList=tList0)
                _,eVecs_0l2=yu.GEVP(t2pt[1],-dtt0,tList=tList0)
                _,eVecs_1l1=yu.GEVP(t2pt[2],-dtt0,tList=tList1)
                _,eVecs_1l2=yu.GEVP(t2pt[3],-dtt0,tList=tList1)

                eVecs0_ltni=np.array([eVecs_0l1,eVecs_0l2])
                eVecs1_ltni=np.array([eVecs_1l1,eVecs_1l2])
                if case=='noGEVP':
                    dt2pt0_lij,dt2pt1_lij,dt3pt00_ijlm,dt3pt11_ijlm,dt3pt01_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1,np.zeros((1,1,2,2))+1,np.zeros((1,1,2,2))+1)
                else:
                    dt2pt0_lij,dt2pt1_lij,dt3pt01_ijlm=get_dt(eVecs0_ltni,eVecs1_ltni,ensL,opsL0,opsL1)
                    _,_,dt3pt00_ijlm=get_dt(eVecs0_ltni,eVecs0_ltni,ensL,opsL0,opsL0)
                    _,_,dt3pt11_ijlm=get_dt(eVecs1_ltni,eVecs1_ltni,ensL,opsL1,opsL1)
                
                t2pt0_ltij=np.array([t2pt[0],t2pt[1]])
                t2pt0_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pt0_ltij,dt2pt0_lij)),axis=0)
                
                t2pt1_ltij=np.array([t2pt[2],t2pt[3]])
                t2pt1_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pt1_ltij,dt2pt1_lij)),axis=0)

                def t_get2(i,i_FF,FF):
                    opasL,opbsL=FF2opabsL[FF]
                    AInv={'0':AInv0,'1':AInv1}[opasL]; BInv={'0':AInv0,'1':AInv1}[opbsL]
                    dt3pt_ijlm={'00':dt3pt00_ijlm,'11':dt3pt11_ijlm,'01':dt3pt01_ijlm}[opasL+opbsL]
                    res=0
                    for i_pip,pip in enumerate(FF2pips(ensL,FF)):
                        prefactor,insert,proj=pip
                        proj=proj2mat[proj]
                        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                        res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_FF][i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                    return res
                t=[t2pt0_t,t2pt1_t]+[ t_get2(i,i_FF,FF) for i_FF,FF in enumerate(FFs) for i,tf in enumerate(tfs)]
            
                t=yu.prefactorDeep(t,1)
                return t
                                      
            datmin=3
            pars0=np.concatenate([ens2p0[(ens,pt0,case)],ens2p0[(ens,pt1,case)],\
                yu.sl(('fitSep',ensL,'GA_00_zz',case,diagsL,datmin))[0][[4]],\
                    yu.sl(('fitSep',ensL,'GA_00_zz',case,diagsL,datmin))[0][[5]],yu.sl(('fitSep',ensL,'GA_11_tz',case,diagsL,datmin))[0][[5]],\
                        yu.sl(('fitSep',ensL,'GA_11_zz',case,diagsL,datmin))[0][[5]],yu.sl(('fitSep',ensL,'GA_11_xx',case,diagsL,datmin))[0][[5]],\
                            [1],\
                                yu.sl(('fitSep',ensL,'G5_01',case,diagsL,datmin))[0][[8,9,10]], yu.sl(('fitSep',ensL,'GA_01',case,diagsL,datmin))[0][[8,9,10]], \
                    yu.sl(('fitSep',ensL,'GP_01_tz',case,diagsL,datmin))[0][[8,9,10]], yu.sl(('fitSep',ensL,'GP_01_zz',case,diagsL,datmin))[0][[9,10]]      
                ])
            
            for datmin in datmins:
                print(ensL,diagsL,case,datmin,end='                              \r')
                datmin_3pt_tf=datmin_3pt_tci=datmin_3pt_tcf=datmin
                tfs_2pt0_fit=np.arange(ens2tfRange[(ens,pt0,case)][0],ens2tfRange[(ens,pt0,case)][1])
                tfs_2pt1_fit=np.arange(ens2tfRange[(ens,pt1,case)][0],ens2tfRange[(ens,pt1,case)][1])
                
                tfs_3pt_fit=[tf for tf in tfs if tf>=datmin_3pt_tf and tf+1>(datmin_3pt_tci+datmin_3pt_tcf)]
                tcs_3pt00_fit={tf:np.arange(datmin_3pt_tci,tf//2+1) for tf in tfs_3pt_fit}
                tcs_3ptJ5_fit={tf:np.arange(datmin_3pt_tci,tf//2) for tf in tfs_3pt_fit}
                tcs_3pt01_fit={tf:np.arange(datmin_3pt_tci,tf+1-datmin_3pt_tcf) for tf in tfs_3pt_fit}
                
                def FF2tcs_3pt_fit(FF):
                    opasL,opbsL=FF2opabsL[FF]
                    tcs_3pt_fit={'00':tcs_3pt00_fit,'11':tcs_3pt00_fit,'01':tcs_3pt01_fit}[opasL+opbsL]
                    if FF is 'J5_11':
                        tcs_3pt_fit=tcs_3ptJ5_fit
                    return tcs_3pt_fit
                
                def func(dat):
                    t=func_base(dat)
                    t[0]=t[0][tfs_2pt0_fit]
                    t[1]=t[1][tfs_2pt1_fit]
                    for i_pips in range(len(FFs)):
                        FF=FFs[i_pips]
                        tcs_3pt_fit=FF2tcs_3pt_fit(FF)
                        for i,tf in enumerate(tfs):
                            t[i_pips*len(tfs)+i+2]=t[i_pips*len(tfs)+i+2][tcs_3pt_fit[tf]-1]
                    return t
                def fitfunc(E0_0,c0_0,dE1_0,rc1_0, E0_1,c0_1,dE1_1,rc1_1, g_gA, ra01_GA_00_zz,ra01_GA_11_tz,ra01_GA_11_zz,ra01_GA_11_xx, ra01_J5_11,\
                    g_G5,ra01_G5_01,ra10_G5_01, g_GA,ra01_GA_01,ra10_GA_01, g_GP,ra01_GP_01_tz,ra10_GP_01_tz, ra01_GP_01_zz,ra10_GP_01_zz):
                    t_2pt0=func_C2pt_2st(tfs_2pt0_fit,E0_0,c0_0,dE1_0,rc1_0)
                    t_2pt1=func_C2pt_2st(tfs_2pt1_fit,E0_1,c0_1,dE1_1,rc1_1)
                    t_3pt=[func_C3pt_2st(tf,FF2tcs_3pt_fit(FF)[tf],E0a,E0b,gcc,dE1a,dE1b,ra01,ra10,0) for FF, E0a,dE1a, E0b,dE1b, gcc,ra01,ra10 in \
                        [('GA_00_zz', E0_0,dE1_0, E0_0,dE1_0, g_gA*np.sqrt(c0_0*c0_0), ra01_GA_00_zz,ra01_GA_00_zz), 
                         ('GA_11_tz', E0_1,dE1_1, E0_1,dE1_1, g_gA*np.sqrt(c0_1*c0_1), ra01_GA_11_tz,ra01_GA_11_tz), 
                         ('GA_11_zz', E0_1,dE1_1, E0_1,dE1_1, g_gA*np.sqrt(c0_1*c0_1), ra01_GA_11_zz,ra01_GA_11_zz), 
                         ('GA_11_xx', E0_1,dE1_1, E0_1,dE1_1, g_gA*np.sqrt(c0_1*c0_1), ra01_GA_11_xx,ra01_GA_11_xx), 
                         ('J5_11', E0_1,dE1_1, E0_1,dE1_1, 0, ra01_J5_11,-ra01_J5_11),
                         ('G5_01', E0_0,dE1_0, E0_1,dE1_1, g_G5*np.sqrt(c0_0*c0_1), ra01_G5_01,ra10_G5_01), 
                         ('GA_01', E0_0,dE1_0, E0_1,dE1_1, g_GA*np.sqrt(c0_0*c0_1), ra01_GA_01,ra10_GA_01), 
                         ('GP_01_tz', E0_0,dE1_0, E0_1,dE1_1, g_GP*np.sqrt(c0_0*c0_1), ra01_GP_01_tz,ra10_GP_01_tz), 
                         ('GP_01_zz', E0_0,dE1_0, E0_1,dE1_1, g_GP*np.sqrt(c0_0*c0_1), ra01_GP_01_zz,ra10_GP_01_zz), 
                         ] for tf in tfs_3pt_fit]
                    t=[t_2pt0,t_2pt1]+t_3pt
                    return t
                def estimator(pars): 
                    E0_0,c0_0,dE1_0,rc1_0, E0_1,c0_1,dE1_1,rc1_1, g_gA, ra01_GA_00_zz,ra01_GA_11_tz,ra01_GA_11_zz,ra01_GA_11_xx, ra01_J5_11,\
                    g_G5,ra01_G5_01,ra10_G5_01, g_GA,ra01_GA_01,ra10_GA_01, g_GP,ra01_GP_01_tz,ra10_GP_01_tz, ra01_GP_01_zz,ra10_GP_01_zz=pars
                    rPCAC_5A=(lat.amu/lat.ZP[0])/lat.amN * g_G5/g_GA; rPCAC_PA=t_aQ2/(4*mN**2) * g_GP/g_GA
                    return [rPCAC_5A,rPCAC_PA,g_G5,g_GA,g_GP,E0_0,E0_1,g_gA]
                res=yu.fit(dat,func,fitfunc,estimator=estimator,pars0=pars0,sl_key=('fitPCAC',ensL,case,diagsL,datmin))

# for ensL in ensLs:    
#     run(ensL)

In [ ]:
# plot fitSep v.s. fitSep_a11

diagsL='all'
FF='GA_00_zz'
ren=FF2ren(ensL,FF)

ensL='48'
ens=L2ens[ensL]
updateEns(ens)
fig, axs = yu.getFigAxs(1,2,sharex=True,sharey=True)
yu.addColHeader(axs,[r'w/o $a_{11}$',r'with $a_{11}$'],fontsize=35)

opasL,opbsL=FF2opabsL[FF]
ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
pta,ofas=ptofas; ptb,ofbs=ptofbs

for icol in [0,1]:
    axs[irow,icol].set_xlim(xlim_fit)
    axs[irow,icol].set_xticks(xticks_fit)
    axs[irow,icol].set_xlabel(xlabel_fit,fontsize=35)
    
axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
# axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
axs[irow,0].set_ylabel(r'$g^{u-d}_A$',fontsize=35)

# if ensL in ['48'] and FF in FF2simone:
#     t_mean,t_err=simone[FF2simone[FF]]
#     axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
#     if FF not in ['J5_11','rPCAC_1_Ptz','rPCAC_1_Pzz']:
#         axs[irow,1].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

# if FF in ['J5_11']:
#     axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
# elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
#     axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
# elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
#     for icol in range(2):
#         axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')

for case in ['noGEVP','GEVP']:
    color={'noGEVP':'b','GEVP':'r'}[case]
    mfc={'noGEVP':'white','GEVP':None}[case]
    shift_sgn={'noGEVP':-1,'GEVP':1}[case]
    xshift={'noGEVP':-0.005,'GEVP':0.005}[case]
    
    icol=0
    fits=[]
    for datmin in datmins:
        res=yu.sl(('fitSep',ensL,FF,case,diagsL,datmin))
        if res==None:
            continue
        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
        
        ind=4 if pta==ptb else 8

        tMean=[pars_mean[ind]]; tErr=[pars_err[ind]]
        tMean,tErr=errRen(tMean,tErr,ren)
        axs[irow,icol].errorbar([datmin*xUnit+xshift],tMean,tErr,color=color,mfc=mfc)
        ylim=ylimDic[(ensL,FF)]
        corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.05,'GEVP':0.92}[case])
        axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,corY),color=color,size=18)
        # axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,tMean+shift_sgn*(tErr+shift_chi2)),color=color,size=18)
        
    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
    tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
    tMean,tErr=errRen(tMean,tErr,ren)
    axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
    
    icol=1
    fits=[]
    for datmin in datmins:
        res=yu.sl(('fitSep_a11',ensL,FF,case,diagsL,datmin))
        if res==None:
            continue
        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
        
        ind=4 if pta==ptb else 8

        tMean=[pars_mean[ind]]; tErr=[pars_err[ind]]
        tMean,tErr=errRen(tMean,tErr,ren)
        axs[irow,icol].errorbar([datmin*xUnit+xshift],tMean,tErr,color=color,mfc=mfc)
        ylim=ylimDic[(ensL,FF)]
        corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.05,'GEVP':0.92}[case])
        axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,corY),color=color,size=18)
        # axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,tMean+shift_sgn*(tErr+shift_chi2)),color=color,size=18)
        
    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
    tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
    tMean,tErr=errRen(tMean,tErr,ren)
    axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
for icol in range(2):
    axs[irow,icol].locator_params(axis='y', nbins=4)
    axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 

plt.tight_layout()
plt.savefig('fig/3pt_raw_sep'+'_'+ensL+'_a11.pdf')
plt.close()

In [ ]:
colorMap_rainbow={('noGEVP',10):'skyblue',('noGEVP',12):'royalblue',('noGEVP',14):'b',('GEVP',10):'tomato',('GEVP',12):'indianred',('GEVP',14):'red'}
colorMap_rainbow={('noGEVP',10):'green',('noGEVP',12):'blue',('noGEVP',14):'purple',('GEVP',10):'yellow',('GEVP',12):'orange',('GEVP',14):'red'}

# plot: ratio & seperate fit
oneColClass=['J5_11']+FFinPCAC
def run(ensL,FFs,label):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(len(FFs),2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [3, 1]})
    
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=35)
        for icol in [1]:
            axs[irow,icol].set_xlim(xlim_fit)
            axs[irow,icol].set_xticks(xticks_fit)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_fit,fontsize=35)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=35)
        # axs[irow,1].set_ylabel(FF2label_charge[FF],fontsize=35)
        # axs[irow,1].yaxis.set_label_position("right")
        
        if ensL in ['48'] and FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            if FF not in ['J5_11','rPCAC_1_Ptz','rPCAC_1_Pzz']:
                axs[irow,1].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        if FF in ['J5_11']:
            axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
        elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
            axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
        elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
            for icol in range(2):
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(['all']):
            icol=i_diags*2+0
            for case in ['noGEVP','GEVP']:
                color={'noGEVP':'b','GEVP':'r'}[case]
                mfc={'noGEVP':'white','GEVP':None}[case]
                xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                res=yu.sl(('ratio',ensL,FF,case,diagsL))
                mean,err,cov=res
                
                if FF in FFinPCAC:
                    for i_tf,tf in enumerate(tfs):
                        tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                        tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                        tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                        tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                        terr=np.sqrt(np.diag(tcov))
                        tMean=tmean;tErr=terr 
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                    continue
                
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
        
        if FF not in oneColClass:
            shift_chi2=(ylimDic[(ensL,FF)][1]-ylimDic[(ensL,FF)][0])/15
            for i_diags,diagsL in enumerate(['all']):
                icol=i_diags*2+1
                for case in ['noGEVP','GEVP']:
                    color={'noGEVP':'b','GEVP':'r'}[case]
                    mfc={'noGEVP':'white','GEVP':None}[case]
                    shift_sgn={'noGEVP':-1,'GEVP':1}[case]
                    xshift={'noGEVP':-0.005,'GEVP':0.005}[case]
                    
                    fits=[]
                    for datmin in datmins:
                        res=yu.sl(('fitSep',ensL,FF,case,diagsL,datmin))
                        if res==None:
                            continue
                        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                        
                        ind=4 if pta==ptb else 8

                        tMean=[pars_mean[ind]]; tErr=[pars_err[ind]]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol].errorbar([datmin*xUnit+xshift],tMean,tErr,color=color,mfc=mfc)
                        ylim=ylimDic[(ensL,FF)]
                        corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.05,'GEVP':0.92}[case])
                        axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,corY),color=color,size=18)
                        # axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,tMean+shift_sgn*(tErr+shift_chi2)),color=color,size=18)
                        
                    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                    tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
        # else:
        #     for icol in [1,3]:
        #         axs[irow,icol].axis('off')

    for irow in range(len(FFs)):
        for icol in range(2):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_sep_'+label+'_'+ensL+'.pdf')
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ensLs:
    run(ensL,['GSb+_00','GSb+_11'],'S')
    run(ensL,['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'],'AAAA')
    # run(ensL,['J5_11'],'J')
    run(ensL,['G5_01','GA_01','GP_01_tz','GP_01_zz'],'5APP')
    # run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
    print()

In [ ]:
# plot: ratio & seperate fit
oneColClass=['J5_11']+FFinPCAC
def run(ensL,FFs,label):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(len(FFs),2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [4, 0]})
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=35)
        for icol in [1]:
            axs[irow,icol].set_xlim(xlim_fit)
            axs[irow,icol].set_xticks(xticks_fit)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_fit,fontsize=35)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=35)
        # t_ax=axs[irow,0].twinx()
        # t_ax.set_ylim(ylimDic[(ensL,FF)])
        # t_ax.set_ylabel(FF2label_charge[FF],fontsize=35)

        if ensL in ['48'] and FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            if FF not in ['J5_11','rPCAC_1_Ptz','rPCAC_1_Pzz']:
                axs[irow,1].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        if FF in ['J5_11']:
            axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
        elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
            axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
        elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
            for icol in range(2):
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(['all']):
            icol=i_diags*2+0
            for case in ['noGEVP','GEVP']:
                color={'noGEVP':'b','GEVP':'r'}[case]
                mfc={'noGEVP':'white','GEVP':None}[case]
                xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                mean,err,cov=yu.sl(('ratio',ensL,FF,case,diagsL))
                
                if FF in FFinPCAC:
                    for i_tf,tf in enumerate(tfs):
                        tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                        tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                        tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                        tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                        terr=np.sqrt(np.diag(tcov))
                        tMean=tmean;tErr=terr 
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                    continue
                
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  

    for irow in range(len(FFs)):
        for icol in [1]:
            axs[irow,icol].axis('off')
        
        for icol in range(2):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_sep_'+label+'_'+ensL+'.pdf')
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ensLs:
    run(ensL,['J5_11'],'J')
    run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
    print()

In [ ]:

# plot: ratio & seperate fit (no-J & all)
oneColClass=['J5_11']+FFinPCAC
def run(ensL,FFs,label):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(len(FFs),4,sharex='col',sharey='row', gridspec_kw={'width_ratios': [3, 1, 3, 1]})
    
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0,2]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=35)
        for icol in [1,3]:
            axs[irow,icol].set_xlim(xlim_fit)
            axs[irow,icol].set_xticks(xticks_fit)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_fit,fontsize=35)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=35)
        
        if ensL in ['48'] and FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            axs[irow,2].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            if FF not in ['J5_11','rPCAC_1_Ptz','rPCAC_1_Pzz']:
                axs[irow,1].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
                axs[irow,3].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        if FF in ['J5_11']:
            axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
            axs[irow,2].axhline(y=0,color='black',linestyle = '--', marker='')
        elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
            axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
            axs[irow,2].axhline(y=1,color='black',linestyle = '--', marker='')
        elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
            for icol in range(4):
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(diagsLs):
            icol=i_diags*2+0
            for case in ['noGEVP','GEVP']:
                color={'noGEVP':'b','GEVP':'r'}[case]
                mfc={'noGEVP':'white','GEVP':None}[case]
                xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                res=yu.sl(('ratio',ensL,FF,case,diagsL))
                mean,err,cov=res
                
                if FF in FFinPCAC:
                    for i_tf,tf in enumerate(tfs):
                        tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                        tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                        tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                        tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                        terr=np.sqrt(np.diag(tcov))
                        tMean=tmean;tErr=terr 
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                    continue
                
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
        
        if FF not in oneColClass:
            shift_chi2=(ylimDic[(ensL,FF)][1]-ylimDic[(ensL,FF)][0])/15
            for i_diags,diagsL in enumerate(diagsLs):
                icol=i_diags*2+1
                for case in ['noGEVP','GEVP']:
                    color={'noGEVP':'b','GEVP':'r'}[case]
                    mfc={'noGEVP':'white','GEVP':None}[case]
                    shift_sgn={'noGEVP':-1,'GEVP':1}[case]
                    xshift={'noGEVP':-0.005,'GEVP':0.005}[case]
                    
                    fits=[]
                    for datmin in datmins:
                        res=yu.sl(('fitSep',ensL,FF,case,diagsL,datmin))
                        if res==None:
                            continue
                        pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                        fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                        
                        ind=4 if pta==ptb else 8

                        tMean=[pars_mean[ind]]; tErr=[pars_err[ind]]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol].errorbar([datmin*xUnit+xshift],tMean,tErr,color=color,mfc=mfc)
                        ylim=ylimDic[(ensL,FF)]
                        corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.05,'GEVP':0.92}[case])
                        axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,corY),color=color,size=18)
                        # axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit+xshift,tMean+shift_sgn*(tErr+shift_chi2)),color=color,size=18)
                        
                    pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
                    tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
        # else:
        #     for icol in [1,3]:
        #         axs[irow,icol].axis('off')

    for irow in range(len(FFs)):
        for icol in range(4):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_sep_'+label+'_'+ensL+'_noJ_all.pdf')
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ['48']:
    # run(ensL,['GSb+_00','GSb+_11'],'S')
    # run(ensL,['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'],'AAAA')
    # run(ensL,['J5_11'],'J')
    run(ensL,['G5_01','GA_01','GP_01_tz','GP_01_zz'],'5APP')
    # run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
    print()

In [ ]:
# plot: ratio & seperate fit (no-J & all)
oneColClass=['J5_11']+FFinPCAC
def run(ensL,FFs,label):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(len(FFs),4,sharex='col',sharey='row', gridspec_kw={'width_ratios': [4, 0, 4 ,0]})
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0,2]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=35)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=35)
        
        if ensL in ['48'] and FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            axs[irow,2].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        if FF in ['J5_11']:
            axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
            axs[irow,2].axhline(y=0,color='black',linestyle = '--', marker='')
        elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
            axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
            axs[irow,2].axhline(y=1,color='black',linestyle = '--', marker='')
        elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
            for icol in range(4):
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(diagsLs):
            icol=i_diags*2+0
            for case in ['noGEVP','GEVP']:
                color={'noGEVP':'b','GEVP':'r'}[case]
                mfc={'noGEVP':'white','GEVP':None}[case]
                xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                mean,err,cov=yu.sl(('ratio',ensL,FF,case,diagsL))
                
                if FF in FFinPCAC:
                    for i_tf,tf in enumerate(tfs):
                        tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                        tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                        tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                        tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                        terr=np.sqrt(np.diag(tcov))
                        tMean=tmean;tErr=terr 
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                    continue
                
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[i_tf];tErr=err[i_tf]
                    tMean,tErr=errRen(tMean,tErr,ren)
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  

    for irow in range(len(FFs)):
        for icol in [1,3]:
            axs[irow,icol].axis('off')
        
        for icol in range(4):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_sep_'+label+'_'+ensL+'_noJ_all.pdf')
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ['48']:
    # run(ensL,['J5_11'],'J')
    run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
    print()

In [ ]:
# plot: ratio (R vs R_std)
oneColClass=['J5_11']+FFinPCAC

fsize=25
def run(ensL,FFs,label,subQ=True):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(len(FFs),2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [3, 3]})
    yu.addColHeader(axs,[r'$R$',r'$R^{\rm std}$'],fontsize=fsize)
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0,1]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=fsize)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=fsize)
        
        # if ensL in ['48'] and FF in FF2simone:
        #     t_mean,t_err=simone[FF2simone[FF]]
        #     axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        #     axs[irow,1].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        #     # axs[irow,2].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        # if FF in ['J5_11']:
        #     axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
        #     axs[irow,1].axhline(y=0,color='black',linestyle = '--', marker='')
        #     # axs[irow,2].axhline(y=0,color='black',linestyle = '--', marker='')
        # elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
        #     axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
        #     axs[irow,1].axhline(y=1,color='black',linestyle = '--', marker='')
        #     # axs[irow,2].axhline(y=1,color='black',linestyle = '--', marker='')
        # elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
        #     for icol in range(2):
        #         axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(['all']):
            for icol,Itype in enumerate(['R','Rstd']):
                for case in ['noGEVP','GEVP']:
                    color={'noGEVP':'b','GEVP':'r'}[case]
                    mfc={'noGEVP':'white','GEVP':None}[case]
                    xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                    base='ratio' if Itype is 'R' else 'ratio_std'
                    res=yu.sl((base,ensL,FF,case,diagsL))
                    # print(Itype,res)
                    mean,err,cov=res
                    
                    if FF in FFinPCAC:
                        for i_tf,tf in enumerate(tfs):
                            tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                            tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                            tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                            tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                            terr=np.sqrt(np.diag(tcov))
                            tMean=tmean;tErr=terr 
                            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                        continue
                    
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
        

    for irow in range(len(FFs)):
        for icol in range(2):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=fsize) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_'+label+'_'+ensL+'_R_Rstd.pdf')
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ['24']:
    for subQ in [True]:
        # run(ensL,['GP_01_zz'],'GP_01_zz',subQ)
        # run(ensL,['G5_01'],'G5_01',subQ)
        run(ensL,['GA_01'],'GA_01',subQ)
        # run(ensL,['GSb+_00','GSb+_11'],'S')
        # run(ensL,['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'],'AAAA')
        # run(ensL,['J5_11'],'J',subQ)
        # run(ensL,['G5_01','GA_01','GP_01_tz','GP_01_zz'],'5APP')
        # run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
        print()


In [ ]:

# plot: ratio (Id vs Ip vs Ipp)
oneColClass=['J5_11']+FFinPCAC
def run(ensL,FFs,label,subQ=True):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(len(FFs),3,sharex='col',sharey='row', gridspec_kw={'width_ratios': [3, 3, 3]})
    yu.addColHeader(axs,[r'$R_d$',r'$R^\prime$',r'$R^{\prime\prime}$'],fontsize=35)
    
    for irow,FF in enumerate(FFs):
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0,1,2]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==len(FFs)-1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=35)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=35)
        
        # if ensL in ['48'] and FF in FF2simone:
        #     t_mean,t_err=simone[FF2simone[FF]]
        #     axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        #     axs[irow,1].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        #     axs[irow,2].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        # if FF in ['J5_11']:
        #     axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
        #     axs[irow,1].axhline(y=0,color='black',linestyle = '--', marker='')
        #     axs[irow,2].axhline(y=0,color='black',linestyle = '--', marker='')
        # elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
        #     axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
        #     axs[irow,1].axhline(y=1,color='black',linestyle = '--', marker='')
        #     axs[irow,2].axhline(y=1,color='black',linestyle = '--', marker='')
        # elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
        #     for icol in range(3):
        #         axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(['all']):
            for icol,Itype in enumerate(['Id','Ip','Ipp']):
                for case in ['noGEVP','GEVP']:
                    color={'noGEVP':'b','GEVP':'r'}[case]
                    mfc={'noGEVP':'white','GEVP':None}[case]
                    xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                    base='ratio_' if subQ else 'ratio_0sub_'
                    res=yu.sl((base+Itype,ensL,FF,case,diagsL))
                    mean,err,cov=res
                    
                    if FF in FFinPCAC:
                        for i_tf,tf in enumerate(tfs):
                            tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                            tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                            tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                            tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                            terr=np.sqrt(np.diag(tcov))
                            tMean=tmean;tErr=terr 
                            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                        continue
                    
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
        

    for irow in range(len(FFs)):
        for icol in range(3):
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_'+label+'_'+ensL+('_Id_Ip_Ipp.pdf' if subQ else '_Id_Ip_Ipp_0sub.pdf'))
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ['48']:
    for subQ in [True]:
        run(ensL,['GP_01_zz'],'GP_01_zz',subQ)
        # run(ensL,['GSb+_00','GSb+_11'],'S')
        # run(ensL,['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'],'AAAA')
        # run(ensL,['J5_11'],'J',subQ)
        # run(ensL,['G5_01','GA_01','GP_01_tz','GP_01_zz'],'5APP')
        # run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
        print()


In [ ]:

# plot: ratio (Id vs Ip vs Ipp) 0fit, 0sub vs sub
oneColClass=['J5_11']+FFinPCAC
def run(ensL,FFs,label):
    ens=L2ens[ensL]
    updateEns(ens)
    
    fig, axs = yu.getFigAxs(2,3,sharex=True,sharey=True, gridspec_kw={'width_ratios': [3, 3, 3]})
    yu.addColHeader(axs,[r'$R_d$',r'$R^\prime$',r'$R^{\prime\prime}$'],fontsize=35)
    
    for irow,subQ in enumerate([True,False]):
        FF=FFs[0]
        opasL,opbsL=FF2opabsL[FF]
        ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        xUnit=lat.a
        
        ren=FF2ren(ensL,FF)
        
        for icol in [0,1,2]:
            axs[irow,icol].set_xlim(xlim_rainbow)
            axs[irow,icol].set_xticks(xticks_rainbow)
            if irow==1:
                axs[irow,icol].set_xlabel(xlabel_rainbow,fontsize=35)
        axs[irow,0].set_ylim(ylimDic[(ensL,FF)])
        # axs[irow,0].text(.05, .95, FF2label[FF], ha='left', va='top', size=30, transform=axs[irow,0].transAxes)
        axs[irow,0].set_ylabel(FF2label_ratio[FF],fontsize=35)
        axs[irow,0].set_ylabel(['Subtracted','NOT subtracted'][irow],fontsize=35)
        
        if ensL in ['48'] and FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,0].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            axs[irow,1].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            axs[irow,2].fill_between(xlim_rainbow,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)

        if FF in ['J5_11']:
            axs[irow,0].axhline(y=0,color='black',linestyle = '--', marker='')
            axs[irow,1].axhline(y=0,color='black',linestyle = '--', marker='')
            axs[irow,2].axhline(y=0,color='black',linestyle = '--', marker='')
        elif FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
            axs[irow,0].axhline(y=1,color='black',linestyle = '--', marker='')
            axs[irow,1].axhline(y=1,color='black',linestyle = '--', marker='')
            axs[irow,2].axhline(y=1,color='black',linestyle = '--', marker='')
        elif FF in ['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'] and ensL in ['48']:
            for icol in range(3):
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
        
        for i_diags,diagsL in enumerate(['all']):
            for icol,Itype in enumerate(['Id','Ip','Ipp']):
                for case in ['noGEVP','GEVP']:
                    color={'noGEVP':'b','GEVP':'r'}[case]
                    mfc={'noGEVP':'white','GEVP':None}[case]
                    xshift={'noGEVP':-0.003,'GEVP':0.003}[case]
                    base='ratio_0fit_'
                    res=yu.sl((base+Itype,subQ,ensL,FF,case,diagsL))
                    mean,err,cov=res
                    
                    if FF in FFinPCAC:
                        for i_tf,tf in enumerate(tfs):
                            tmean=np.hstack([mean[i_tf],mean[len(tfs)+i_tf],1])
                            tcov=np.block([[cov[i_tf][i_tf],cov[i_tf][len(tfs)+i_tf]],[cov[len(tfs)+i_tf][i_tf],cov[len(tfs)+i_tf][len(tfs)+i_tf]]])
                            tcov=np.block([[tcov,np.zeros((len(tcov),1))],[np.zeros((1,len(tcov))),(ren[1]/ren[0])**2]])
                            tmean,tcov=yu.propagateError(lambda x:x[:tf-1]*x[-1]+x[tf-1:-1],tmean,tcov)
                            terr=np.sqrt(np.diag(tcov))
                            tMean=tmean;tErr=terr 
                            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
                        continue
                    
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        tMean,tErr=errRen(tMean,tErr,ren)
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit+xshift,tMean,tErr,color=colorMap_rainbow[(case,tf)],fmt=['s','d','o'][i_tf],mfc=mfc)  
        

    for irow in range(2):
        for icol in range(3):
            axs[irow,icol].locator_params(axis='x', nbins=3)
            axs[irow,icol].locator_params(axis='y', nbins=4)
            axs[irow,icol].tick_params(axis='both', which='major', labelsize=35) 
    
    plt.tight_layout()
    plt.savefig('fig/3pt_raw_'+label+'_'+ensL+('_Id_Ip_Ipp_0fit.pdf'))
    plt.close()
    print(ensL,label,end='\t') 
    
for ensL in ['48']:
    # run(ensL,['GP_01_zz'],'GP_01_zz')
    # run(ensL,['GSb+_00','GSb+_11'],'S')
    # run(ensL,['GA_00_zz','GA_11_tz','GA_11_zz','GA_11_xx'],'AAAA')
    run(ensL,['J5_11'],'J')
    # run(ensL,['G5_01','GA_01','GP_01_tz','GP_01_zz'],'5APP')
    # run(ensL,['rPCAC_1_Ptz','rPCAC_1_Pzz'],'rr')
    print()


In [ ]:
# plot: A5
fsize=15
def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)

    xUnit=lat.a
    
    fig, axs = yu.getFigAxs(1,1,sharex=True,sharey='row',Lrow=6.4*1.2*0.4,Lcol=4.8*1.2)
    # fig.suptitle(L2T_ens[ensL]+', A5')
    # yu.addRowHeader(axs,['gA'])
    # yu.addColHeader(axs,['no-J','all'])
    
    if ensL in ['48']:
        for irow,FF in enumerate(['GA_00_zz']):
            t_mean,t_err=simone[FF2simone[FF]]
            for icol in [0]:
                axs[irow,icol].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
    
    def func(mean,cov):
        tmean,tcov=yu.extendMeanCov(mean,cov,[(1,lat.ZA[1]/lat.ZA[0])])
        def tfunc(x):
            g_gA,E0a,E0b, rZA=x
            return [g_gA*rZA]
        tmean,tcov=yu.propagateError(tfunc,tmean,tcov)
        return tmean,np.sqrt(np.diag(tcov)),tcov
    
    for icol,diagsL in enumerate(['all']):
        for case in ['noGEVP','GEVP']:
            fits=[]
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            shift_sgn={'noGEVP':-1,'GEVP':1}[case]
            xshift={'noGEVP':-0.005,'GEVP':0.005}[case]
            for datmin in datmins:
                res=yu.sl(('fitA5',ensL,case,diagsL,datmin))
                if res is None:
                    continue
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                tmean,terr,tcov=func(pars_mean,pars_cov)
                fits.append((tmean,terr,chi2R_mean,Ndof))
                for irow,ind in enumerate([0]):
                    tMean=tmean[ind]; tErr=terr[ind]
                    axs[irow,icol].errorbar(datmin*xUnit+xshift,tMean,tErr,color=color,mfc=mfc)
                    FF=['GA_00_zz'][irow]
                    shift_chi2=(ylimDic[(ensL,FF)][1]-ylimDic[(ensL,FF)][0])/10
                    ylim=ylimDic[(ensL,FF)]
                    corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.1,'GEVP':0.82}[case])
                    axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,corY),color=color,size=14)
                    axs[irow,icol].set_ylim(ylimDic[(ensL,FF)])
            
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)     
            for irow,ind in enumerate([0]):
                tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
                axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                print(ensL,diagsL,case,tMean,tErr)
                
    # axs[0,0].text(.05, .95, r'$g_A^{u-d}$', ha='left', va='top', size=20, transform=axs[irow,0].transAxes)
    irow=0; icol=0
    axs[irow,icol].set_xlim(xlim_fit)
    axs[irow,icol].set_xticks(xticks_fit)
    axs[irow,icol].tick_params(axis='both', which='major', labelsize=fsize)
    for irow,FF in enumerate(['GA_00_zz']):
        axs[irow,icol].set_ylabel(r'$g_A^{u-d}$',fontsize=fsize)
    
    for icol in [0]:
        axs[0,icol].set_xlabel(xlabel_fit,fontsize=fsize)
    
    plt.tight_layout()
    plt.savefig('fig/3pt_A5_'+ensL+'.pdf')
    plt.close()
    # print(ensL,end='\t') 
    
for ensL in ensLs:
    run(ensL)

In [ ]:
# plot: PCAC01

fsize=15
def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)

    xUnit=lat.a
    
    fig, axs = yu.getFigAxs(4,1,sharex=True,sharey='row',Lrow=6.4*1.2*0.4,Lcol=4.8*1.2)
    # fig.suptitle(L2T_ens[ensL]+', PCAC01')
    # yu.addRowHeader(axs,['G5','GP','GA','rPCAC'])
    # yu.addColHeader(axs,['no-J','all'])
    
    for irow,FF in enumerate(['G5_01','GP_01_zz','GA_01','rPCAC_1_Pzz']):
        t_mean,t_err=simone[FF2simone[FF]]
        for icol in [0]:
            if ensL in ['48']:
                axs[irow,icol].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            if FF in ['rPCAC_1_Pzz']:
                axs[irow,icol].axhline(y=1,color='black',linestyle = '--', marker='')
    
    def func(mean,cov):
        tmean,tcov=yu.extendMeanCov(mean,cov,[(1,lat.ZSbyZP[1]/lat.ZSbyZP[0]),(1,lat.ZS[1]/lat.ZS[0]),(1,lat.ZA[1]/lat.ZA[0])])
        def tfunc(x):
            rPCAC_5A,rPCAC_PA,g_5,g_A,g_P,E0a,E0b, rZSbyZP,rZS,rZA=x
            return [rPCAC_5A*rZSbyZP+rPCAC_PA, g_5*rZS, g_A*rZA, g_P*rZA]
        tmean,tcov=yu.propagateError(tfunc,tmean,tcov)
        return tmean,np.sqrt(np.diag(tcov)),tcov
    
    for icol,diagsL in enumerate(['all']):
        for case in ['noGEVP','GEVP']:
            fits=[]
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            shift_sgn={'noGEVP':-1,'GEVP':1}[case]
            xshift={'noGEVP':-0.005,'GEVP':0.005}[case]
            for datmin in datmins:
                res=yu.sl(('fitPCAC01',ensL,case,diagsL,datmin))
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                tmean,terr,tcov=func(pars_mean,pars_cov)
                fits.append((tmean,terr,chi2R_mean,Ndof))
                for irow,ind in enumerate([1,3,2,0]):
                    tMean=tmean[ind]; tErr=terr[ind]
                    axs[irow,icol].errorbar(datmin*xUnit+xshift,tMean,tErr,color=color,mfc=mfc)
                    FF=['G5_01','GP_01_zz','GA_01','rPCAC_1_Pzz'][irow]
                    shift_chi2=(ylimDic[(ensL,FF)][1]-ylimDic[(ensL,FF)][0])/10
                    ylim=ylimDic[(ensL,FF)]
                    corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.1,'GEVP':0.88}[case])
                    axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,corY),color=color,size=14)
                    # axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean+shift_sgn*(tErr+shift_chi2)),color=color,size=18)
                    axs[irow,icol].set_ylim(ylimDic[(ensL,FF)])
            
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            for irow,ind in enumerate([1,3,2,0]):
                tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
                axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                
    # for irow in range(4):
    #     axs[irow,0].text(.05, .95, [r'$G^{u-d}_5(Q_1^2)$',r'$G^{u-d}_P(Q_1^2)$',r'$G^{u-d}_A(Q_1^2)$',r'$r^{u-d}_{\mathrm{PCAC}}(Q_1^2)$'][irow], ha='left', va='top', size=20, transform=axs[irow,0].transAxes)
    irow=0; icol=0
    axs[irow,icol].set_xlim(xlim_fit)
    axs[irow,icol].set_xticks(xticks_fit)
    for irow,FF in enumerate(['G5_01','GP_01_zz','GA_01','rPCAC_1_Pzz']):
        axs[irow,icol].set_ylabel([r'$G^{u-d}_5(Q_1^2)$',r'$G^{u-d}_P(Q_1^2)$',r'$G^{u-d}_A(Q_1^2)$',r'$r^{u-d}_{\mathrm{PCAC}}(Q_1^2)$'][irow],fontsize=fsize)
        axs[irow,icol].tick_params(axis='both', which='major', labelsize=fsize)
    for icol in [0]:
        axs[-1,icol].set_xlabel(xlabel_fit,fontsize=fsize)


    plt.tight_layout()
    plt.savefig('fig/3pt_PCAC01_'+ensL+'.pdf')
    plt.close()
    print(ensL,end='\t') 
    
for ensL in ensLs:
    run(ensL)

In [ ]:
# plot: PCAC

def run(ensL):
    ens=L2ens[ensL]
    updateEns(ens)

    xUnit=lat.a
    
    fig, axs = yu.getFigAxs(5,1,sharex=True,sharey='row',Lrow=6.4*1.2*0.4,Lcol=4.8*1.2)
    # fig.suptitle(L2T_ens[ensL]+', PCAC')
    # yu.addRowHeader(axs,['G5','GP','GA','rPCAC','gA'])
    # yu.addColHeader(axs,['no-J','all'])
    
    FFs=['G5_01','GP_01_zz','GA_01','rPCAC_1_Pzz','GA_00_zz']
    for irow,FF in enumerate(FFs):
        t_mean,t_err=simone[FF2simone[FF]]
        for icol in [0]:
            if ensL in ['48']:
                axs[irow,icol].fill_between(xlim_fit,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            if FF in ['rPCAC_1_Pzz']:
                axs[irow,icol].axhline(y=1,color='black',linestyle = '--', marker='')
            if FF in ['GA_00_zz'] and ensL in ['48']:
                axs[irow,icol].axhline(y=gA_exp,color='black',linestyle = '--', marker='')
    
    def func(mean,cov):
        tmean,tcov=yu.extendMeanCov(mean,cov,[(1,lat.ZSbyZP[1]/lat.ZSbyZP[0]),(1,lat.ZS[1]/lat.ZS[0]),(1,lat.ZA[1]/lat.ZA[0])])
        def tfunc(x):
            rPCAC_5A,rPCAC_PA,g_5,g_A,g_P,E0a,E0b,g_gA, rZSbyZP,rZS,rZA=x
            return [rPCAC_5A*rZSbyZP+rPCAC_PA, g_5*rZS, g_A*rZA, g_P*rZA, g_gA*rZA]
        tmean,tcov=yu.propagateError(tfunc,tmean,tcov)
        return tmean,np.sqrt(np.diag(tcov)),tcov
    inds=[1,3,2,0,4]
    
    for icol,diagsL in enumerate(['all']):
        for case in ['noGEVP','GEVP']:
            fits=[]
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            shift_sgn={'noGEVP':-1,'GEVP':1}[case]
            for datmin in datmins:
                res=yu.sl(('fitPCAC',ensL,case,diagsL,datmin))
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                tmean,terr,tcov=func(pars_mean,pars_cov)
                fits.append((tmean,terr,chi2R_mean,Ndof))
                for irow,ind in enumerate(inds):
                    tMean=tmean[ind]; tErr=terr[ind]
                    axs[irow,icol].errorbar(datmin*xUnit,tMean,tErr,color=color,mfc=mfc)
                    FF=FFs[irow]
                    ylim=ylimDic[(ensL,FF)]
                    corY=ylim[0]+(ylim[1]-ylim[0])* ({'noGEVP':0.1,'GEVP':0.85}[case])
                    axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,corY),color=color,size=14)
                    # axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean+shift_sgn*(tErr+shift_chi2)),color=color,size=18)
                    axs[irow,icol].set_ylim(ylimDic[(ensL,FF)])
            
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            for irow,ind in enumerate(inds):
                tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
                axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
                # print(ensL,diagsL,case,FFs[irow],tMean,tErr)
                
    # for irow in range(5):
    #     axs[irow,0].text(.05, .95, [r'$G^{u-d}_5(Q_1^2)$',r'$G^{u-d}_P(Q_1^2)$',r'$G^{u-d}_A(Q_1^2)$',r'$r^{u-d}_{\mathrm{PCAC}}(Q_1^2)$',r'$g^{u-d}_A$'][irow], ha='left', va='top', size=20, transform=axs[irow,0].transAxes)
    irow=0; icol=0
    axs[irow,icol].set_xlim(xlim_fit)
    axs[irow,icol].set_xticks(xticks_fit)
    for irow,FF in enumerate(['G5_01','GP_01_zz','GA_01','rPCAC_1_Pzz','GA_00_zz']):
        axs[irow,icol].set_ylabel([r'$G^{u-d}_5(Q_1^2)$',r'$G^{u-d}_P(Q_1^2)$',r'$G^{u-d}_A(Q_1^2)$',r'$r^{u-d}_{\mathrm{PCAC}}(Q_1^2)$',r'$g^{u-d}_A$'][irow])
    for icol in [0]:
        axs[-1,icol].set_xlabel(xlabel_fit)

    plt.tight_layout()
    plt.savefig('fig/3pt_PCAC_'+ensL+'.pdf')
    plt.close()
    print(ensL,end='\t') 
    
# for ensL in ensLs:
#     run(ensL)